# <b> <span style="color:white">Electricity Sector Data Streaming & Analysis</span></b>


# <b> <span style="color:white">GROUP 04</span></b>


| Name                   | SID       | Unikey   |
| ---------------------- | --------- | -------- |
| Putu Eka Udiyani Putri | 550067302 | pput0940 |
| Rengga Firmandika      | 550126632 | rfir0117 |
| Vincentius Ansel Suppa | 550206406 | vsup0468 |


## <b> <span style="color:orange">0. Configuration and Import Required Libraries</span></b>


**Quick start:**
1. Project structure:
   
   <pre>
   Assignment2_Tut07_G04/
   ├── Assignment_2.ipynb      # main notebook
   └── requirements.txt        # list of required libraries to run the notebook
   </pre>

   Ensure your working directory is writable.

2. Create venv & install exact dependencies<br/>
   `python -m venv .venv`<br/>
   Windows: `.\.venv\Scripts\activate` | macOS/Linux: `source .venv/bin/activate`<br/>
   `python -m pip install --upgrade pip`<br/>
   `pip install -r requirements.txt`

3. Copy `.env.template` to `.env` file, replace `your_api_key` with your actual API key. 

4. Run the full pipeline<br/>


Import all the required libraries first.


In [7]:
from dotenv import load_dotenv
import os

import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import json
import math
import glob
from pathlib import Path
import paho.mqtt.client as mqtt
import sys
import duckdb
from thefuzz import fuzz, process


## <b> <span style="color:orange">1. Data Retrieval</span></b>


In [2]:
DB_PATH = "energy_dw.duckdb"

con = duckdb.connect(DB_PATH) 
con.execute("INSTALL spatial")
con.execute("LOAD spatial")
result = con.execute("SELECT * FROM duckdb_extensions() WHERE installed").fetchall()
print(result)
con.close()

[('core_functions', True, True, '(BUILT-IN)', 'Core function library', [], 'v1.4.1', 'STATICALLY_LINKED', ''), ('icu', True, True, '(BUILT-IN)', 'Adds support for time zones and collations using the ICU library', [], 'v1.4.1', 'STATICALLY_LINKED', ''), ('json', True, True, '(BUILT-IN)', 'Adds support for JSON operations', [], 'v1.4.1', 'STATICALLY_LINKED', ''), ('parquet', True, True, '(BUILT-IN)', 'Adds support for reading and writing parquet files', [], 'v1.4.1', 'STATICALLY_LINKED', ''), ('spatial', True, True, 'C:\\Users\\Rengga\\.duckdb\\extensions\\v1.4.1\\windows_amd64\\spatial.duckdb_extension', 'Geospatial extension that adds support for working with spatial data and functions', [], 'a6a607f', 'REPOSITORY', 'core')]


In [29]:
con = duckdb.connect(DB_PATH)

# Check first rows
assignment1_facility_data = con.execute("""
            SELECT DISTINCT
                f.year,
                dfa.facility_name,
                dfu.fuel_type,
                dfu.fuel_category,
                dfu.is_renewable,
                dge.latitude,
                dge.longitude,
                dge.state_code,
                dge.geo_resolution
            FROM fact_nger_facility f
            JOIN dim_facility dfa ON dfa.facility_id=f.facility_id
            JOIN dim_fuel dfu ON dfu.fuel_key=f.fuel_key
            JOIN dim_geo dge ON dge.geo_id=f.geo_id
            WHERE f.grid='NEM'
            AND f.year=2023
        """).fetchdf()

con.close()

In [30]:
assignment1_facility_data

,year,facility_name,fuel_type,fuel_category,is_renewable,latitude,longitude,state_code,geo_resolution
0,2023,Hallett (Stage 2) Wind Farm,Wind,WIND,True,-30.534367,135.630121,SA,state
1,2023,Broadwater Power Plant,Wood,BIO,True,-31.875984,147.286949,NSW,state
2,2023,The DPESS Trust,Battery,STORAGE,False,-31.875984,147.286949,NSW,state
3,2023,Moranbah North CMM Power Station,Waste Coal Mine Gas,GAS,False,-22.164678,144.584490,QLD,state
4,2023,Crookwell 3 Windfarm,Wind,WIND,True,-31.875984,147.286949,NSW,state
...,...,...,...,...,...,...,...,...,...
304,2023,Portland Wind Farms,Wind,WIND,True,-36.598610,144.678005,VIC,state
305,2023,Clarke Creek Wind Farm,Wind,WIND,True,-22.164678,144.584490,QLD,state
306,2023,Numurkah Project Pty Ltd,Solar,SOLAR,True,-36.598610,144.678005,VIC,state
307,2023,Suntop Solar Farm,Solar,SOLAR,True,-31.875984,147.286949,NSW,state


In [12]:
# basic configs
API_KEY = os.getenv("OPENELECTRICITY_API_KEY")
API_KEY = API_KEY.strip().strip('"').strip("'")  
BASE_URL = "https://api.openelectricity.org.au/v4/"
HEADERS = {
        "Authorization": f"Bearer {API_KEY}",
        "Accept": "application/json",
    }


# function to fetch data
def fetch_data_from_API(endpoint: str, query_params: dict): 
    try:
        response = requests.get(f"{BASE_URL}{endpoint}", headers=HEADERS, params=query_params)
        
        print(f"Response status: {response.status_code}")
        print(f"Response url: {response.url}")
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"API Error {response.status_code}: {response.text}")
            print(f"Response headers: {dict(response.headers)}")

            try:
                error_json = response.json()
                print(f"Error details: {error_json}")
            except:
                print("Could not parse error response as JSON")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

# helper function to save dataset
def save_dataset(df: pd.DataFrame, out_csv_path: str):
	out_path = Path(out_csv_path)
	out_path.parent.mkdir(parents=True, exist_ok=True)
	df.to_csv(out_path, index=False)
	print(f"Saved: {out_path}")

### <b> <span style="color:pink">1.1 Get All Facilities in NEM Region</span></b>


In [13]:
# set endpoint and params
ENDPOINT = "facilities/"
PARAMS = {
    'network_id': 'NEM',
}

# fetch facilities data
facilities = fetch_data_from_API(endpoint=ENDPOINT, query_params=PARAMS)
facilities_df = pd.json_normalize(facilities['data'])
facilities_df.head()

Response status: 200
Response url: https://api.openelectricity.org.au/v4/facilities/?network_id=NEM


,code,name,network_id,network_region,description,units,updated_at,created_at,location.lat,location.lng
0,ADP,Adelaide Desalination,NEM,SA1,"<p>The Adelaide Desalination plant (ADP), form...","[{'code': 'ADPPV1', 'fueltech_id': 'solar_util...",2025-08-05T06:08:12Z,2023-10-18T04:34:30Z,-35.096948,138.484061
1,ALDGASF,Aldoga,NEM,QLD1,<p>The Aldoga Solar Farm will be approximately...,"[{'code': 'ALDGASF1', 'fueltech_id': 'solar_ut...",2025-03-25T00:52:44Z,2025-01-31T04:19:33Z,-23.839544,151.084900
2,AMCORGR,Amcor Glass,NEM,SA1,<p></p>,"[{'code': 'AMCORGR', 'fueltech_id': 'distillat...",2023-10-18T04:34:32Z,2023-10-18T04:34:32Z,-34.882663,138.577975
3,ANGASTON,Angaston,NEM,SA1,<p>Angaston Power Station is a diesel-powered ...,"[{'code': 'ANGAS1', 'fueltech_id': 'distillate...",2025-09-07T01:53:13Z,2023-10-18T04:34:32Z,-34.503948,139.024296
4,APS,Anglesea,NEM,VIC1,<p>The Anglesea Power Station was a brown coal...,"[{'code': 'APS', 'fueltech_id': 'coal_brown', ...",2024-11-04T00:41:34Z,2023-10-18T04:34:32Z,-38.389031,144.180589


In [41]:
name_to_code = facilities_df.set_index('name')['code'].to_dict()

def get_best_match(facility_name, choices, threshold=80):
    """Find best matching name and return its code"""
    result = process.extractOne(facility_name, choices, scorer=fuzz.token_sort_ratio)
    if result and result[1] >= threshold:  # result[1] is the score
        return name_to_code[result[0]]
    return None

# Apply fuzzy matching
assignment1_facility_data['code'] = assignment1_facility_data['facility_name'].apply(
    lambda x: get_best_match(x, name_to_code.keys(), threshold=60)
)

In [42]:
assignment1_facility_data[assignment1_facility_data['code'].notnull()]

,year,facility_name,fuel_type,fuel_category,is_renewable,latitude,longitude,state_code,geo_resolution,code
1,2023,Broadwater Power Plant,Wood,BIO,True,-31.875984,147.286949,NSW,state,BWTR1
3,2023,Moranbah North CMM Power Station,Waste Coal Mine Gas,GAS,False,-22.164678,144.584490,QLD,state,MBAHNTH
4,2023,Crookwell 3 Windfarm,Wind,WIND,True,-31.875984,147.286949,NSW,state,CROOKWF3
9,2023,Metz Solar Farm,Solar,SOLAR,True,-31.875984,147.286949,NSW,state,OAKEY1SF
10,2023,PARKES SOLAR FARM PTY LTD,Solar,SOLAR,True,-31.875984,147.286949,NSW,state,OAKEY1SF
...,...,...,...,...,...,...,...,...,...,...
302,2023,MOURA SOLAR FARM SPV PTY LTD,Solar,SOLAR,True,-22.164678,144.584490,QLD,state,OAKEY1SF
304,2023,Portland Wind Farms,Wind,WIND,True,-36.598610,144.678005,VIC,state,WRWF1
305,2023,Clarke Creek Wind Farm,Wind,WIND,True,-22.164678,144.584490,QLD,state,CLRKCWF
307,2023,Suntop Solar Farm,Solar,SOLAR,True,-31.875984,147.286949,NSW,state,WRSF1


In [8]:
# Step 0 - Trial - Eka

import duckdb, pandas as pd

# connect to the DuckDB database created in Assignment 1
con = duckdb.connect("energy_dw.duckdb", read_only=True)

print("Tables in the database:")
print(con.execute("SHOW TABLES;").df())

# peek likely facility tables to understand the schema
for t in ["fact_nger_facility", "dim_facility", "dim_geo", "dim_fuel"]:
    try:
        print(f"\nPreview of table '{t}':")
        display(con.execute(f"SELECT * FROM {t} LIMIT 5").df())
    except Exception as e:
        pass


Tables in the database:
                  name
0      dim_corporation
1         dim_facility
2             dim_fuel
3              dim_geo
4         dim_industry
5            dim_stage
6    fact_abs_industry
7  fact_abs_population
8     fact_cer_project
9   fact_nger_facility

Preview of table 'fact_nger_facility':


,facility_id,year,corporation_id,fuel_key,electricity_gj,electricity_mwh,emissions_1,emissions_2,emissions_total,emissions_intensity,grid_connected,grid,geo_id
0,1,2014,1,1,567719.0,157700.0,19.0,293.0,312,0.0,ON,NEM,2316593
1,1,2014,1,1,567719.0,157700.0,19.0,293.0,312,0.0,ON,NEM,2316593
2,1,2014,1,1,567719.0,157700.0,19.0,293.0,312,0.0,ON,NEM,2316593
3,1,2014,1,1,567719.0,157700.0,19.0,293.0,312,0.0,ON,NEM,2316593
4,1,2014,1,1,567719.0,157700.0,19.0,293.0,312,0.0,ON,NEM,2316593



Preview of table 'dim_facility':


,facility_id,facility_name
0,1,Gunning Wind Farm
1,2,Royalla Solar Farm
2,3,Waubra Wind Farm
3,4,Banimboola Hydro
4,5,Bayswater Power Station



Preview of table 'dim_geo':


,geo_id,latitude,longitude,state_code,display_name,postcode,min_lat,max_lat,min_lon,max_lon,geo_resolution,osm_type,geocoder
0,289547296,-20.729005,139.493224,QLD,"Mount Isa Post Office, Isa Street, Mount Isa C...",4825.0,-20.729055,-20.728955,139.493174,139.493274,postcode,node,nominatim
1,5519705,-33.965003,150.801103,NSW,"Leppington, Sydney, Camden Council, New South ...",2179.0,-34.004331,-33.948324,150.759408,150.841382,exact,relation,nominatim
2,2918464324,-33.719717,150.892220,NSW,"Quakers Hillside Care Community, Hambledon Rd,...",2763.0,-33.719767,-33.719667,150.892170,150.892270,exact,node,nominatim
3,22963243,-37.817580,144.969976,VIC,"Ian Potter Centre: NGV Australia, Russell Stre...",3008.0,-37.818067,-37.817012,144.969603,144.970339,postcode,way,nominatim
4,15268146,-27.541981,153.079185,QLD,"Hillsong Brisbane Campus, 16, Rover Street, Mo...",4074.0,-27.542259,-27.541120,153.078875,153.079691,postcode,relation,nominatim



Preview of table 'dim_fuel':


,fuel_key,fuel_type,fuel_category,is_renewable
0,1,Wind,WIND,True
1,2,Solar,SOLAR,True
2,3,Hydro,HYDRO,True
3,4,Black Coal,COAL,False
4,5,Gas,GAS,False


In [9]:
# Pull both sources (from DB and from API) - Trial - Eka

# Pull facility master directly from DuckDB (joined dimensions)
query = """
SELECT
    f.facility_id        AS facility_code,
    d.facility_name,
    g.latitude,
    g.longitude,
    g.state_code         AS region,
    fuel.fuel_type,
    fuel.fuel_category,
    fuel.is_renewable
FROM fact_nger_facility AS f
LEFT JOIN dim_facility AS d ON f.facility_id = d.facility_id
LEFT JOIN dim_geo      AS g ON f.geo_id      = g.geo_id
LEFT JOIN dim_fuel     AS fuel ON f.fuel_key = fuel.fuel_key
WHERE g.latitude IS NOT NULL
GROUP BY 1,2,3,4,5,6,7,8
"""

facility_master = con.execute(query).df()
print("Facility master from DB:", facility_master.shape)
facility_master.head()


Facility master from DB: (692, 8)


,facility_code,facility_name,latitude,longitude,region,fuel_type,fuel_category,is_renewable
0,791,Chinchilla BESS,-22.164678,144.584490,QLD,Battery,STORAGE,False
1,63,Broadwater Power Plant,-31.875984,147.286949,NSW,Wood,BIO,True
2,64,Condong Power Plant,-31.875984,147.286949,NSW,Wood,BIO,True
3,212,The Drop Hydro,-31.875984,147.286949,NSW,Hydro,HYDRO,True
4,514,Yaloak South Wind Farm,-36.598610,144.678005,VIC,Wind,WIND,True


In [14]:
# Build a clean API dataframe for matching - Trial - Eka
facilities_api = facilities_df.rename(columns={
    "code": "facility_code",
    "name": "facility_name",
    "location.lat": "lat_api",
    "location.lng": "lng_api",
    "network_region": "region_api",
})[["facility_code","facility_name","region_api","lat_api","lng_api"]]

print("API facilities:", facilities_api.shape)
facilities_api.head(5)


API facilities: (514, 5)


,facility_code,facility_name,region_api,lat_api,lng_api
0,ADP,Adelaide Desalination,SA1,-35.096948,138.484061
1,ALDGASF,Aldoga,QLD1,-23.839544,151.084900
2,AMCORGR,Amcor Glass,SA1,-34.882663,138.577975
3,ANGASTON,Angaston,SA1,-34.503948,139.024296
4,APS,Anglesea,VIC1,-38.389031,144.180589


In [15]:
# Another match by facility_code - Trial - Eka

import re
import pandas as pd

def normalise_name(s: str) -> str:
    if pd.isna(s): return ""
    s = s.lower()
    s = re.sub(r"\b(p/l|pty|ltd|limited|power\s*station|pp|plant|facility|unit|station|co|company)\b", " ", s)
    s = re.sub(r"[\(\)\[\]\.,/\\\-_'’]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# 4a) API: normalised name
api2 = facilities_api.copy()
api2["name_norm"]   = api2["facility_name"].apply(normalise_name)
api2["region_api"]  = api2["region_api"].astype(str).str.upper().str.strip()

# 4b) DB: normalised name + map state_code -> NEM region code (NSW->NSW1, dll.)
REGION_MAP = {"NSW":"NSW1","VIC":"VIC1","QLD":"QLD1","SA":"SA1","TAS":"TAS1"}
db2 = facility_master.copy()
db2["name_norm"]    = db2["facility_name"].apply(normalise_name)
db2["region_db"]    = db2["region"].astype(str).str.upper().str.strip().map(REGION_MAP)

# (opsional) buang yang di luar NEM (region_db NaN)
db2 = db2[~db2["region_db"].isna()].copy()

print("API rows:", len(api2), "| DB rows:", len(db2))
api2.head(3), db2.head(3)


API rows: 514 | DB rows: 495


(  facility_code          facility_name region_api    lat_api     lng_api  \
 0           ADP  Adelaide Desalination        SA1 -35.096948  138.484061   
 1       ALDGASF                 Aldoga       QLD1 -23.839544  151.084900   
 2       AMCORGR            Amcor Glass        SA1 -34.882663  138.577975   
 
                name_norm  
 0  adelaide desalination  
 1                 aldoga  
 2            amcor glass  ,
    facility_code           facility_name   latitude   longitude region  \
 0            791         Chinchilla BESS -22.164678  144.584490    QLD   
 1             63  Broadwater Power Plant -31.875984  147.286949    NSW   
 2             64     Condong Power Plant -31.875984  147.286949    NSW   
 
   fuel_type fuel_category  is_renewable         name_norm region_db  
 0   Battery       STORAGE         False   chinchilla bess      QLD1  
 1      Wood           BIO          True  broadwater power      NSW1  
 2      Wood           BIO          True     condong power    

In [16]:
# Attempt 1: Exact match by name_norm + region, with proper progress tracking and ID fallback

import pandas as pd
import numpy as np

def _ensure_match_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure tracking columns exist on api2."""
    if "matched_facility_id" not in df.columns:
        df["matched_facility_id"] = pd.Series([np.nan] * len(df), dtype="object")
    if "match_strategy" not in df.columns:
        df["match_strategy"] = pd.Series([np.nan] * len(df), dtype="object")
    return df

def _progress(a2: pd.DataFrame, label: str):
    """Print compact progress of matching status."""
    total = len(a2)
    matched = a2["matched_facility_id"].notna().sum()
    remaining = total - matched
    pct = (matched / total * 100.0) if total else 0.0
    print(f"[{label}] total rows: {total}")
    print(f"[{label}] matched rows: {matched}")
    print(f"[{label}] remaining unmatched: {remaining}")
    print(f"[{label}] progress: {pct:.2f}%")

# Ensure tracking columns exist
api2 = _ensure_match_columns(api2)

# Work only on rows not yet matched
candidates = api2[api2["matched_facility_id"].isna()].copy()
candidates = candidates.reset_index().rename(columns={"index": "index_api"})

# Attempt 1: exact merge by name_norm + region
merged_name = candidates.merge(
    db2,
    left_on=["name_norm", "region_api"],
    right_on=["name_norm", "region_db"],
    how="left",
    suffixes=("_api", "_db")
)

# Determine which rows are matched
# If facility_id_db exists, use it to decide; otherwise fall back to facility_name_db
has_id_col = "facility_id_db" in merged_name.columns
is_matched = merged_name["facility_id_db"].notna() if has_id_col else merged_name["facility_name_db"].notna()

matched_name = merged_name[is_matched].copy()
unmatched_name = merged_name[~is_matched].copy()

# ---- Fallback ID selection: facility_id_db -> facility_code_db -> facility_name_db ----
id_candidates = [c for c in ["facility_id_db", "facility_code_db", "facility_name_db"] if c in matched_name.columns]
if not id_candidates:
    # If none of the ID-like columns exist, we cannot record progress; keep informative print and skip writing
    print("No ID-like columns found in db2 (facility_id_db / facility_code_db / facility_name_db).")
else:
    # Build a single chosen ID column by taking first non-null across available candidates
    matched_name["__chosen_id"] = matched_name[id_candidates].bfill(axis=1).iloc[:, 0]
    # Some rows could still be NaN if all candidates null; keep only rows with a chosen id
    matched_name = matched_name[matched_name["__chosen_id"].notna()].copy()

    # Apply matches back into api2 using index_api
    if not matched_name.empty:
        api_idx = matched_name["index_api"].values
        api2.loc[api_idx, "matched_facility_id"] = matched_name["__chosen_id"].astype(str).values
        api2.loc[api_idx, "match_strategy"] = "exact_name_region"

# Attempt-specific summary
print(f"Matched by name+region in this attempt: {len(matched_name)}")
print(f"Unmatched after name+region in this attempt: {len(unmatched_name)}")

# Global cumulative progress
_progress(api2, label="Attempt 1: exact name+region")

# Optional: quick peek
display(matched_name.head(5))
display(unmatched_name.head(5))


Matched by name+region in this attempt: 23
Unmatched after name+region in this attempt: 492
[Attempt 1: exact name+region] total rows: 514
[Attempt 1: exact name+region] matched rows: 22
[Attempt 1: exact name+region] remaining unmatched: 492
[Attempt 1: exact name+region] progress: 4.28%


,index_api,facility_code_api,facility_name_api,region_api,lat_api,lng_api,name_norm,matched_facility_id,match_strategy,facility_code_db,facility_name_db,latitude,longitude,region,fuel_type,fuel_category,is_renewable,region_db,__chosen_id
18,18,BARCALDN,Barcaldine,QLD1,-23.552171,145.314851,barcaldine,NaN,NaN,447.0,Barcaldine Power Station Facility,-22.164678,144.584490,QLD,Gas,GAS,False,QLD1,447.0
19,19,BARCSF,Barcaldine,QLD1,-23.547333,145.318972,barcaldine,NaN,NaN,447.0,Barcaldine Power Station Facility,-22.164678,144.584490,QLD,Gas,GAS,False,QLD1,447.0
21,21,BARRON,Barron Gorge,QLD1,-16.850770,145.647049,barron gorge,NaN,NaN,325.0,Barron Gorge Power Station,-22.164678,144.584490,QLD,Hydro,HYDRO,True,QLD1,325.0
39,39,BOLIVAR,Bolivar,SA1,-34.495482,138.387641,bolivar,NaN,NaN,757.0,Bolivar Power Station,-30.534367,135.630121,SA,Gas,GAS,False,SA1,757.0
40,40,BOLIVPS,Bolivar,SA1,-34.777200,138.582600,bolivar,NaN,NaN,757.0,Bolivar Power Station,-30.534367,135.630121,SA,Gas,GAS,False,SA1,757.0


,index_api,facility_code_api,facility_name_api,region_api,lat_api,lng_api,name_norm,matched_facility_id,match_strategy,facility_code_db,facility_name_db,latitude,longitude,region,fuel_type,fuel_category,is_renewable,region_db
0,0,ADP,Adelaide Desalination,SA1,-35.096948,138.484061,adelaide desalination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,ALDGASF,Aldoga,QLD1,-23.839544,151.084900,aldoga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,AMCORGR,Amcor Glass,SA1,-34.882663,138.577975,amcor glass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,ANGASTON,Angaston,SA1,-34.503948,139.024296,angaston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,APS,Anglesea,VIC1,-38.389031,144.180589,anglesea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Attempt 2: Fuzzy match by name_norm within the same region

import pandas as pd
import numpy as np

try:
    from rapidfuzz import fuzz, process
except Exception as e:
    raise RuntimeError("RapidFuzz is required. Install it via: pip install rapidfuzz") from e

# ---------- helpers ----------
def _ensure_match_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure tracking columns exist on api2."""
    if "matched_facility_id" not in df.columns:
        df["matched_facility_id"] = pd.Series([np.nan] * len(df), dtype="object")
    if "match_strategy" not in df.columns:
        df["match_strategy"] = pd.Series([np.nan] * len(df), dtype="object")
    return df

def _progress(a2: pd.DataFrame, label: str):
    """Print compact cumulative progress."""
    total = len(a2)
    matched = a2["matched_facility_id"].notna().sum()
    remaining = total - matched
    pct = (matched / total * 100.0) if total else 0.0
    print(f"[{label}] total rows: {total}")
    print(f"[{label}] matched rows: {matched}")
    print(f"[{label}] remaining unmatched: {remaining}")
    print(f"[{label}] progress: {pct:.2f}%")

def _safe_preview(df, desired_cols, n=10, title=None):
    """Display only columns that actually exist to avoid KeyError."""
    if title:
        print(title)
    cols = [c for c in desired_cols if c in df.columns]
    if cols:
        display(df[cols].head(n))
    else:
        display(df.head(n))

# ---------- ensure trackers ----------
api2 = _ensure_match_columns(api2)

# ---------- guards ----------
req_api_cols = ["name_norm", "region_api"]
missing_api = [c for c in req_api_cols if c not in api2.columns]
if missing_api:
    raise ValueError(f"api2 is missing required columns: {missing_api}")

req_db_cols = ["name_norm", "region_db"]
missing_db = [c for c in req_db_cols if c not in db2.columns]
if missing_db:
    raise ValueError(f"db2 is missing required columns: {missing_db}")

# Choose an identifier to store into matched_facility_id (priority order)
db_id_candidates = [c for c in ["facility_id", "facility_code", "facility_name"] if c in db2.columns]
if not db_id_candidates:
    raise ValueError("db2 does not have any ID-like columns: facility_id / facility_code / facility_name")
CHOSEN_DB_ID_COL = db_id_candidates[0]
print(f"[Attempt 2] Using '{CHOSEN_DB_ID_COL}' from db2 as the match identifier.")

# ---------- parameters ----------
FUZZ_THRESHOLD = 90   # adjust later if needed
SCORER = fuzz.token_set_ratio
STRATEGY_LABEL = f"fuzzy_name_region_tokenset_{FUZZ_THRESHOLD}"

# ---------- build candidates (only rows not yet matched) ----------
cand = api2[api2["matched_facility_id"].isna()].copy()
cand = cand.reset_index().rename(columns={"index": "index_api"})
cand = cand[cand["name_norm"].notna()]  # need a name to compare

# ---------- prepare db reference ----------
db_ref = db2.copy()
db_ref = db_ref[db_ref["name_norm"].notna() & db_ref[CHOSEN_DB_ID_COL].notna()].copy()
db_ref["__chosen_id"] = db_ref[CHOSEN_DB_ID_COL].astype(str)

# Group available choices by region to keep comparisons relevant
region_to_choices = (
    db_ref.groupby("region_db")
          .apply(lambda g: list(zip(g["name_norm"].astype(str).tolist(),
                                    g["__chosen_id"].tolist())))
          .to_dict()
)

# Keep only rows whose region has choices
cand["__has_choices"] = cand["region_api"].map(lambda r: r in region_to_choices)
cand_work = cand[cand["__has_choices"]].copy()

# ---------- fuzzy matching per region ----------
best_ids, best_scores, best_names = [], [], []

for region, g in cand_work.groupby("region_api"):
    choices = region_to_choices.get(region, [])
    if not choices:
        n = len(g)
        best_ids.extend([np.nan]*n)
        best_scores.extend([np.nan]*n)
        best_names.extend([np.nan]*n)
        continue

    choice_names = [nm for nm, _id in choices]
    choice_ids   = [ _id for nm, _id in choices]

    queries = g["name_norm"].astype(str).tolist()
    scores_mat = process.cdist(queries, choice_names, scorer=SCORER, workers=0)
    argmax = scores_mat.argmax(axis=1)
    scores = scores_mat.max(axis=1)

    best_ids.extend([choice_ids[i] for i in argmax])
    best_scores.extend(scores.tolist())
    best_names.extend([choice_names[i] for i in argmax])

cand_work["__best_id"] = best_ids
cand_work["__best_score"] = best_scores
cand_work["__best_name"] = best_names

accepted = cand_work[cand_work["__best_score"] >= FUZZ_THRESHOLD].copy()
rejected = cand_work[cand_work["__best_score"] < FUZZ_THRESHOLD].copy()
no_region_choices = cand[~cand["__has_choices"]]

# ---------- write back to api2 ----------
if not accepted.empty:
    api_idx = accepted["index_api"].values
    api2.loc[api_idx, "matched_facility_id"] = accepted["__best_id"].astype(str).values
    api2.loc[api_idx, "match_strategy"] = STRATEGY_LABEL

# ---------- summaries ----------
print(f"Fuzzy accepted (>= {FUZZ_THRESHOLD}): {len(accepted)}")
print(f"Fuzzy below threshold: {len(rejected)}")
print(f"No region choices: {len(no_region_choices)}")
_progress(api2, label="Attempt 2: fuzzy name within region")

# ---------- safe previews ----------
_safe_preview(
    accepted,
    desired_cols=[
        "index_api","facility_name_api","name_norm","region_api",
        "__best_name","__best_score","matched_facility_id","match_strategy"
    ],
    n=10,
    title="Accepted (top 10)"
)
_safe_preview(
    rejected,
    desired_cols=[
        "index_api","facility_name_api","name_norm","region_api",
        "__best_name","__best_score"
    ],
    n=10,
    title="Rejected (top 10)"
)


[Attempt 2] Using 'facility_code' from db2 as the match identifier.
Fuzzy accepted (>= 90): 256
Fuzzy below threshold: 235
No region choices: 1
[Attempt 2: fuzzy name within region] total rows: 514
[Attempt 2: fuzzy name within region] matched rows: 278
[Attempt 2: fuzzy name within region] remaining unmatched: 236
[Attempt 2: fuzzy name within region] progress: 54.09%
Accepted (top 10)


C:\Users\Rengga\AppData\Local\Temp\ipykernel_19628\253743194.py:80: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: list(zip(g["name_norm"].astype(str).tolist(),


,index_api,name_norm,region_api,__best_name,__best_score,matched_facility_id,match_strategy
0,0,adelaide desalination,SA1,appin csm,100.0,NaN,NaN
1,1,aldoga,QLD1,avonlie solar farm,100.0,NaN,NaN
2,2,amcor glass,SA1,awaba renewable energy,100.0,NaN,NaN
3,3,angaston,SA1,bango wind farm,100.0,NaN,NaN
6,6,ararat,VIC1,beryl solar farm,100.0,NaN,NaN
7,7,avonlie,NSW1,newtricity developments biala,100.0,NaN,NaN
8,8,awaba,NSW1,blayney wind farm,100.0,NaN,NaN
10,10,baking board,QLD1,boco rock wind farm,100.0,NaN,NaN
11,11,bald hills,VIC1,bodangora wind farm,100.0,NaN,NaN
12,12,ballarat,VIC1,bomen solar farm,100.0,NaN,NaN


Rejected (top 10)


,index_api,name_norm,region_api,__best_name,__best_score
4,4,anglesea,VIC1,dubbo solar hub,50.000000
5,5,appin,NSW1,broadwater power,56.000000
9,9,bairnsdale,VIC1,belrose lfg,50.000000
16,16,bankstown sports club,NSW1,essential energy generation broken hill gas tu...,73.333336
17,17,bannerton,VIC1,bomen solar farm,46.666668
18,20,barker inlet,SA1,albury solar,42.105263
21,24,bell bay,TAS1,oakey 1 solar farm,46.153847
22,25,bell bay,TAS1,australand,44.444443
36,41,bomen,NSW1,cullerin wind farm,46.153847
40,45,braemar 2,QLD1,broadwater power,53.846153


In [18]:
# Attempt 3: Exact match by facility code
# Robust to duplicate column names in db2; includes region check, ID fallback, and progress

import pandas as pd
import numpy as np

# ---------- helpers ----------
def _ensure_match_columns(df: pd.DataFrame) -> pd.DataFrame:
    if "matched_facility_id" not in df.columns:
        df["matched_facility_id"] = pd.Series([np.nan] * len(df), dtype="object")
    if "match_strategy" not in df.columns:
        df["match_strategy"] = pd.Series([np.nan] * len(df), dtype="object")
    return df

def _progress(a2: pd.DataFrame, label: str):
    total = len(a2)
    matched = a2["matched_facility_id"].notna().sum()
    remaining = total - matched
    pct = (matched / total * 100.0) if total else 0.0
    print(f"[{label}] total rows: {total}")
    print(f"[{label}] matched rows: {matched}")
    print(f"[{label}] remaining unmatched: {remaining}")
    print(f"[{label}] progress: {pct:.2f}%")

def _pick_first_existing(df: pd.DataFrame, options):
    for c in options:
        if c in df.columns:
            return c
    return None

def _get_single_series(df: pd.DataFrame, colname: str) -> pd.Series:
    """
    Return a single 1D Series for the given colname even if df has duplicated column names.
    If multiple columns share the same name, take the first one.
    """
    # Boolean mask of columns with this name
    mask = df.columns == colname
    count = mask.sum()
    if count == 0:
        raise KeyError(f"Column '{colname}' not found.")
    if count == 1:
        return df.loc[:, colname]
    # If duplicated names, take the first occurrence
    first_idx = np.flatnonzero(mask)[0]
    return df.iloc[:, first_idx]

# ---------- ensure trackers ----------
api2 = _ensure_match_columns(api2)

# ---------- detect code columns ----------
api_code_col = _pick_first_existing(api2, ["facility_code_api", "facility_code", "code"])
db_code_col  = _pick_first_existing(db2,  ["facility_code", "code"])

if api_code_col is None:
    raise ValueError("No facility code column found on api2 (looked for: facility_code_api / facility_code / code).")
if db_code_col is None:
    raise ValueError("No facility code column found on db2 (looked for: facility_code / code).")

# Optional region consistency
api_region_col = "region_api" if "region_api" in api2.columns else None
db_region_col  = "region_db"  if "region_db"  in db2.columns else None
use_region_check = api_region_col is not None and db_region_col is not None

# Choose an identifier to write into matched_facility_id
db_id_candidates = [c for c in ["facility_id", "facility_code", "facility_name"] if c in db2.columns]
if not db_id_candidates:
    raise ValueError("db2 has no identifier columns (expected one of: facility_id / facility_code / facility_name).")
CHOSEN_DB_ID_COL = db_id_candidates[0]

print(f"[Attempt 3] Using '{api_code_col}' (api2) ↔ '{db_code_col}' (db2) for exact code match.")
print(f"[Attempt 3] Writing identifier from db2 column: '{CHOSEN_DB_ID_COL}'.")
if use_region_check:
    print(f"[Attempt 3] Region consistency enabled: '{api_region_col}' ↔ '{db_region_col}'.")

# ---------- candidates: only rows not yet matched and having a code ----------
cand = api2[api2["matched_facility_id"].isna() & api2[api_code_col].notna()].copy()
cand = cand.reset_index().rename(columns={"index": "index_api"})
cand[api_code_col] = cand[api_code_col].astype(str)

# ---------- prepare db reference robustly ----------
db_ref = db2.copy()

# Build single-series keys even if names are duplicated
db_ref["__code_key"] = _get_single_series(db_ref, db_code_col).astype(str)

if use_region_check:
    db_ref["__region_key"] = _get_single_series(db_ref, db_region_col).astype(str)

# Also ensure single-series for chosen ID
id_series = _get_single_series(db_ref, CHOSEN_DB_ID_COL)
db_ref["__chosen_id"] = id_series.astype(str)

# Keep only needed columns and drop rows missing keys/ids
keep_cols = ["__code_key", "__chosen_id"] + (["__region_key"] if use_region_check else [])
db_ref = db_ref[keep_cols].dropna(subset=["__code_key", "__chosen_id"])

# Deduplicate by first occurrence per code (+ region if applicable)
group_keys = ["__code_key"] + (["__region_key"] if use_region_check else [])
db_ref = db_ref.groupby(group_keys, as_index=False).first()

# ---------- merge by code (+ region if available) ----------
left_keys = [api_code_col] + ([api_region_col] if use_region_check else [])
right_keys = ["__code_key"] + (["__region_key"] if use_region_check else [])

merged = cand.merge(
    db_ref,
    left_on=left_keys,
    right_on=right_keys,
    how="left",
    suffixes=("_api", "_db")
)

# Determine matches by presence of identifier
is_matched = merged["__chosen_id"].notna()
matched = merged[is_matched].copy()
unmatched = merged[~is_matched].copy()

# ---------- write back ----------
if not matched.empty:
    api_idx = matched["index_api"].values
    api2.loc[api_idx, "matched_facility_id"] = matched["__chosen_id"].astype(str).values
    api2.loc[api_idx, "match_strategy"] = "exact_facility_code"

# ---------- summaries ----------
print(f"Exact code matches in this attempt: {len(matched)}")
print(f"Unmatched after exact code in this attempt: {len(unmatched)}")
_progress(api2, label="Attempt 3: exact facility code")

# ---------- safe previews ----------
def _safe_preview(df, desired_cols, n=8, title=None):
    if title: print(title)
    cols = [c for c in desired_cols if c in df.columns]
    display(df[cols].head(n) if cols else df.head(n))

_safe_preview(matched, ["index_api", api_code_col, "__chosen_id", "match_strategy"], n=8, title="Matched (sample)")
_safe_preview(unmatched, ["index_api", api_code_col] + ([api_region_col] if use_region_check else []), n=8, title="Unmatched (sample)")


[Attempt 3] Using 'facility_code' (api2) ↔ 'facility_code' (db2) for exact code match.
[Attempt 3] Writing identifier from db2 column: 'facility_code'.
[Attempt 3] Region consistency enabled: 'region_api' ↔ 'region_db'.
Exact code matches in this attempt: 0
Unmatched after exact code in this attempt: 236
[Attempt 3: exact facility code] total rows: 514
[Attempt 3: exact facility code] matched rows: 278
[Attempt 3: exact facility code] remaining unmatched: 236
[Attempt 3: exact facility code] progress: 54.09%
Matched (sample)


,index_api,facility_code,__chosen_id,match_strategy


Unmatched (sample)


,index_api,facility_code,region_api
0,4,APS,VIC1
1,5,APPIN,NSW1
2,9,DEIBDL,VIC1
3,16,BANKSPT,NSW1
4,17,BANNSP,VIC1
5,20,BARKIPS,SA1
6,24,BBDISEL1,TAS1
7,25,BELLBAY,TAS1


In [19]:
# Attempt 4: Geospatial nearest-by-coordinates within region
# Fix: write-back uses positional indices (iloc) after mapping labels -> positions

# import pandas as pd
# import numpy as np

# MAX_KM = 15.0
# EARTH_R_KM = 6371.0
# STRATEGY_LABEL = f"geo_nearest_within_region_le_{int(MAX_KM)}km"

# def _ensure_match_columns(df: pd.DataFrame) -> pd.DataFrame:
#     if "matched_facility_id" not in df.columns:
#         df["matched_facility_id"] = pd.Series([np.nan] * len(df), dtype="object")
#     if "match_strategy" not in df.columns:
#         df["match_strategy"] = pd.Series([np.nan] * len(df), dtype="object")
#     return df

# def _progress(a2: pd.DataFrame, label: str):
#     total = len(a2)
#     matched = a2["matched_facility_id"].notna().sum()
#     remaining = total - matched
#     pct = (matched / total * 100.0) if total else 0.0
#     print(f"[{label}] total rows: {total}")
#     print(f"[{label}] matched rows: {matched}")
#     print(f"[{label}] remaining unmatched: {remaining}")
#     print(f"[{label}] progress: {pct:.2f}%")

# def _pick_first_existing(df: pd.DataFrame, options):
#     for c in options:
#         if c in df.columns:
#             return c
#     return None

# def _haversine_km(lat1, lon1, lat2, lon2):
#     lat1 = np.radians(lat1.astype(float)); lon1 = np.radians(lon1.astype(float))
#     lat2 = np.radians(lat2.astype(float)); lon2 = np.radians(lon2.astype(float))
#     dlat = lat2 - lat1; dlon = lon2 - lon1
#     a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*(np.sin(dlon/2.0)**2)
#     return EARTH_R_KM * 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))

# def _safe_preview(df, desired_cols, n=10, title=None):
#     if title: print(title)
#     cols = [c for c in desired_cols if c in df.columns]
#     display(df[cols].head(n) if cols else df.head(n))

# api2 = _ensure_match_columns(api2)

# # identifier to write back
# db_id_candidates = [c for c in ["facility_id", "facility_code", "facility_name"] if c in db2.columns]
# if not db_id_candidates:
#     raise ValueError("db2 has no identifier columns (facility_id / facility_code / facility_name).")
# CHOSEN_DB_ID_COL = db_id_candidates[0]

# api_region_col = _pick_first_existing(api2, ["region_api","region"])
# db_region_col  = _pick_first_existing(db2,  ["region_db","region"])
# api_lat_col    = _pick_first_existing(api2, ["lat_api","latitude","lat"])
# api_lon_col    = _pick_first_existing(api2, ["lng_api","longitude","lon","long"])
# db_lat_col     = _pick_first_existing(db2,  ["latitude","lat"])
# db_lon_col     = _pick_first_existing(db2,  ["longitude","lon","long"])

# missing = []
# for nm, col in [("api_region",api_region_col),("db_region",db_region_col),
#                 ("api_lat",api_lat_col),("api_lon",api_lon_col),
#                 ("db_lat",db_lat_col),("db_lon",db_lon_col)]:
#     if col is None: missing.append(nm)
# if missing:
#     raise ValueError(f"Attempt 4 needs these columns: {missing}")

# print(f"[Attempt 4r] Coords api2({api_lat_col},{api_lon_col}) ↔ db2({db_lat_col},{db_lon_col}); region: {api_region_col} ↔ {db_region_col}")
# print(f"[Attempt 4r] Writing identifier: '{CHOSEN_DB_ID_COL}', radius ≤ {MAX_KM} km")

# # only unmatched with coords
# cand = api2[api2["matched_facility_id"].isna()].copy()
# cand = cand[cand[api_lat_col].notna() & cand[api_lon_col].notna()].copy()
# cand = cand.reset_index().rename(columns={"index":"index_api"})  # label index of api2

# db_ref = db2.copy()
# db_ref = db_ref[db_ref[db_lat_col].notna() & db_ref[db_lon_col].notna() & db_ref[CHOSEN_DB_ID_COL].notna()].copy()

# accepted_rows_lbl, accepted_ids, accepted_dists = [], [], []
# rejected_rows_lbl, rejected_best = [], []
# no_choices_rows_lbl = []

# for rg, g_api in cand.groupby(cand[api_region_col].astype(str)):
#     g_db = db_ref[db_ref[db_region_col].astype(str)==rg].copy()
#     if g_db.empty:
#         no_choices_rows_lbl.extend(g_api["index_api"].tolist())
#         continue

#     lat_api = g_api[api_lat_col].astype(float).to_numpy()
#     lon_api = g_api[api_lon_col].astype(float).to_numpy()
#     lat_db  = g_db[db_lat_col].astype(float).to_numpy()
#     lon_db  = g_db[db_lon_col].astype(float).to_numpy()

#     best_j = []
#     best_dist = []
#     for i in range(len(g_api)):
#         d = _haversine_km(np.full(len(g_db), lat_api[i]), np.full(len(g_db), lon_api[i]), lat_db, lon_db)
#         j = int(np.argmin(d)); best_j.append(j); best_dist.append(float(d[j]))

#     ids_db = g_db[CHOSEN_DB_ID_COL].astype(str).to_numpy()
#     best_ids_here = [ids_db[j] for j in best_j]

#     idx_lbl = g_api["index_api"].to_numpy()
#     accept_mask = np.array(best_dist) <= MAX_KM

#     accepted_rows_lbl.extend(idx_lbl[accept_mask].tolist())
#     accepted_ids.extend(np.array(best_ids_here)[accept_mask].tolist())
#     accepted_dists.extend(np.array(best_dist)[accept_mask].tolist())

#     rejected_rows_lbl.extend(idx_lbl[~accept_mask].tolist())
#     rejected_best.extend(np.array(best_dist)[~accept_mask].tolist())

# # Map label indices -> positional indices and write via .iloc
# if accepted_rows_lbl:
#     pos_idx = api2.index.get_indexer(accepted_rows_lbl)
#     valid_mask = pos_idx != -1
#     if valid_mask.any():
#         pos_idx_valid = pos_idx[valid_mask]
#         ids_valid = np.array(accepted_ids)[valid_mask]
#         dists_valid = np.array(accepted_dists)[valid_mask]
#         api2.iloc[pos_idx_valid, api2.columns.get_loc("matched_facility_id")] = ids_valid.astype(str)
#         api2.iloc[pos_idx_valid, api2.columns.get_loc("match_strategy")] = STRATEGY_LABEL
#         # distance column
#         if "matched_distance_km" not in api2.columns:
#             api2["matched_distance_km"] = np.nan
#         api2.iloc[pos_idx_valid, api2.columns.get_loc("matched_distance_km")] = dists_valid.astype(float)
#         print(f"[Attempt 4r] Wrote back {valid_mask.sum()} of {len(accepted_rows_lbl)} accepted (index labels mapped).")
#     else:
#         print("[Attempt 4r] Warning: none of the accepted index labels mapped to api2 positions.")
# else:
#     print("[Attempt 4r] No accepted rows to write back.")

# print(f"Geo accepted (≤ {MAX_KM} km): {len(accepted_rows_lbl)}")
# print(f"Geo rejected (best > {MAX_KM} km): {len(rejected_rows_lbl)}")
# print(f"No db choices in region: {len(no_choices_rows_lbl)}")
# _progress(api2, label="Attempt 4r: geo nearest within region")

# # previews
# acc_df = api2.loc[accepted_rows_lbl, [api_region_col, api_lat_col, api_lon_col, "matched_facility_id", "match_strategy", "matched_distance_km"]].copy() if accepted_rows_lbl else pd.DataFrame()
# rej_df = api2.loc[rejected_rows_lbl, [api_region_col, api_lat_col, api_lon_col]].copy() if rejected_rows_lbl else pd.DataFrame()
# _safe_preview(acc_df, [api_region_col, "matched_facility_id", "match_strategy", "matched_distance_km"], 10, "Accepted by geo (top 10)")
# _safe_preview(rej_df, [api_region_col, api_lat_col, api_lon_col], 10, "Rejected by geo (top 10)")


In [20]:
# Attempt 5: Fuzzy match by name_norm without region constraint
# - Optional state_code constraint if both sides have it
# - High threshold to avoid false positives
# - One-to-one resolution: keep highest-score per chosen_id
# - Progress reporting and safe previews

import pandas as pd
import numpy as np

try:
    from rapidfuzz import fuzz, process
except Exception as e:
    raise RuntimeError("RapidFuzz is required. Install it via: pip install rapidfuzz") from e

# ---------- helpers ----------
def _ensure_match_columns(df: pd.DataFrame) -> pd.DataFrame:
    if "matched_facility_id" not in df.columns:
        df["matched_facility_id"] = pd.Series([np.nan] * len(df), dtype="object")
    if "match_strategy" not in df.columns:
        df["match_strategy"] = pd.Series([np.nan] * len(df), dtype="object")
    return df

def _progress(a2: pd.DataFrame, label: str):
    total = len(a2)
    matched = a2["matched_facility_id"].notna().sum()
    remaining = total - matched
    pct = (matched / total * 100.0) if total else 0.0
    print(f"[{label}] total rows: {total}")
    print(f"[{label}] matched rows: {matched}")
    print(f"[{label}] remaining unmatched: {remaining}")
    print(f"[{label}] progress: {pct:.2f}%")

def _safe_preview(df, desired_cols, n=10, title=None):
    if title: print(title)
    cols = [c for c in desired_cols if c in df.columns]
    display(df[cols].head(n) if cols else df.head(n))

def _pick_first_existing(df: pd.DataFrame, options):
    for c in options:
        if c in df.columns:
            return c
    return None

# ---------- ensure trackers ----------
api2 = _ensure_match_columns(api2)

# ---------- guards & column picks ----------
# mandatory names
if "name_norm" not in api2.columns or "name_norm" not in db2.columns:
    raise ValueError("Both api2 and db2 must have a 'name_norm' column for Attempt 5.")

# optional state constraint
api_state_col = _pick_first_existing(api2, ["state_code", "state"])
db_state_col  = _pick_first_existing(db2,  ["state_code", "state"])
use_state_gate = api_state_col is not None and db_state_col is not None

# identifier to store into matched_facility_id
db_id_candidates = [c for c in ["facility_id", "facility_code", "facility_name"] if c in db2.columns]
if not db_id_candidates:
    raise ValueError("db2 does not have an identifier column (facility_id / facility_code / facility_name).")
CHOSEN_DB_ID_COL = db_id_candidates[0]

# ---------- parameters ----------
FUZZ_THRESHOLD = 94  # higher because no region constraint
SCORER = fuzz.token_set_ratio
STRATEGY_LABEL = f"fuzzy_name_no_region_tokenset_{FUZZ_THRESHOLD}"

print(f"[Attempt 5] Using '{CHOSEN_DB_ID_COL}' as identifier; threshold ≥ {FUZZ_THRESHOLD}; "
      f"{'with state gate' if use_state_gate else 'no state gate'}.")

# ---------- build candidates (unmatched only) ----------
cand = api2[api2["matched_facility_id"].isna()].copy()
cand = cand[cand["name_norm"].notna()].copy()
cand = cand.reset_index().rename(columns={"index": "index_api"})

# ---------- prepare db reference ----------
db_ref = db2.copy()
db_ref = db_ref[db_ref["name_norm"].notna() & db_ref[CHOSEN_DB_ID_COL].notna()].copy()
db_ref["__chosen_id"] = db_ref[CHOSEN_DB_ID_COL].astype(str)
if use_state_gate:
    db_ref["__state"] = db_ref[db_state_col].astype(str)

# ---------- fuzzy match ----------
accepted_rows = []
accepted_ids = []
accepted_scores = []
accepted_best_names = []

rejected_rows = []
rejected_scores = []

if use_state_gate:
    # match within same state only
    for st, g in cand.groupby(cand[api_state_col].astype(str)):
        ref = db_ref[db_ref["__state"] == str(st)]
        if ref.empty:
            # no choices for this state; skip
            continue

        choice_names = ref["name_norm"].astype(str).tolist()
        choice_ids   = ref["__chosen_id"].tolist()

        queries = g["name_norm"].astype(str).tolist()
        scores_mat = process.cdist(queries, choice_names, scorer=SCORER, workers=0)
        argmax = scores_mat.argmax(axis=1)
        scores = scores_mat.max(axis=1)

        sel = scores >= FUZZ_THRESHOLD
        accepted_rows.extend(g["index_api"].to_numpy()[sel].tolist())
        accepted_ids.extend(np.array(choice_ids)[argmax][sel].tolist())
        accepted_scores.extend(scores[sel].astype(float).tolist())
        accepted_best_names.extend(np.array(choice_names)[argmax][sel].tolist())

        rejected_rows.extend(g["index_api"].to_numpy()[~sel].tolist())
        rejected_scores.extend(scores[~sel].astype(float).tolist())
else:
    # global choices across all states
    choice_names = db_ref["name_norm"].astype(str).tolist()
    choice_ids   = db_ref["__chosen_id"].tolist()

    queries = cand["name_norm"].astype(str).tolist()
    scores_mat = process.cdist(queries, choice_names, scorer=SCORER, workers=0)
    argmax = scores_mat.argmax(axis=1)
    scores = scores_mat.max(axis=1)

    sel = scores >= FUZZ_THRESHOLD
    accepted_rows.extend(cand["index_api"].to_numpy()[sel].tolist())
    accepted_ids.extend(np.array(choice_ids)[argmax][sel].tolist())
    accepted_scores.extend(scores[sel].astype(float).tolist())
    accepted_best_names.extend(np.array(choice_names)[argmax][sel].tolist())

    rejected_rows.extend(cand["index_api"].to_numpy()[~sel].tolist())
    rejected_scores.extend(scores[~sel].astype(float).tolist())

# ---------- one-to-one resolution: keep highest score per chosen_id ----------
if accepted_rows:
    acc_df = pd.DataFrame({
        "index_api": accepted_rows,
        "__chosen_id": accepted_ids,
        "__score": accepted_scores,
        "__best_name": accepted_best_names
    })
    # keep only the highest score per chosen_id
    acc_df = acc_df.sort_values(["__chosen_id", "__score"], ascending=[True, False])
    acc_df = acc_df.groupby("__chosen_id", as_index=False).first()

    # write back into api2
    api2.loc[acc_df["index_api"].values, "matched_facility_id"] = acc_df["__chosen_id"].astype(str).values
    api2.loc[acc_df["index_api"].values, "match_strategy"] = STRATEGY_LABEL

# ---------- summaries ----------
print(f"Fuzzy accepted (>= {FUZZ_THRESHOLD}): {len(set(accepted_ids)) if accepted_rows else 0}")
print(f"Fuzzy below threshold: {len(rejected_rows)}")
_progress(api2, label="Attempt 5: fuzzy name without region")

# ---------- safe previews ----------
if accepted_rows:
    preview_acc = acc_df.merge(
        api2.reset_index().rename(columns={"index":"index_api"})[["index_api","facility_name_api","name_norm"]]
        if "facility_name_api" in api2.columns else
        api2.reset_index().rename(columns={"index":"index_api"})[["index_api","name_norm"]],
        on="index_api", how="left"
    )
    _safe_preview(preview_acc,
                  ["index_api","facility_name_api","name_norm","__best_name","__score","__chosen_id"],
                  n=10, title="Accepted (top 10)")
else:
    print("No accepted rows to preview.")

rej_df = pd.DataFrame({"index_api": rejected_rows, "__score": rejected_scores})
_safe_preview(rej_df, ["index_api","__score"], n=10, title="Rejected (top 10)")


[Attempt 5] Using 'facility_code' as identifier; threshold ≥ 94; no state gate.
Fuzzy accepted (>= 94): 120
Fuzzy below threshold: 105
[Attempt 5: fuzzy name without region] total rows: 514
[Attempt 5: fuzzy name without region] matched rows: 398
[Attempt 5: fuzzy name without region] remaining unmatched: 116
[Attempt 5: fuzzy name without region] progress: 77.43%
Accepted (top 10)


,index_api,name_norm,__best_name,__score,__chosen_id
0,172,gunning,gunning wind farm,100.0,1
1,103,copeton,copeton hydro,100.0,10
2,5,appin,appin csm,100.0,101
3,53,brooklyn,brooklyn lfg,100.0,108
4,55,browns plains,browns plains lfg,100.0,110
5,130,eildon,eildon hydro,100.0,12
6,148,german creek,german creek cmm,100.0,121
7,205,jacks gully,jacks gully lfg,100.0,125
8,151,glenbawn,glenbawn hydro,100.0,13
9,320,oaky creek 2,oaky creek,100.0,138


Rejected (top 10)


,index_api,__score
0,4,53.333332
1,9,60.000000
2,16,53.658535
3,20,54.545456
4,45,51.851852
5,58,52.631580
6,65,58.823528
7,67,88.888885
8,74,75.000000
9,83,46.666668


In [21]:
# Attempt 6: Hybrid fuzzy + geographic gate (state-aware)
# - Only process rows still unmatched
# - Search candidates within the same state (if available)
# - Accept if fuzzy score >= FUZZ_THRESHOLD AND distance <= MAX_KM
# - Robust write-back and progress reporting

import pandas as pd
import numpy as np

try:
    from rapidfuzz import fuzz, process
except Exception as e:
    raise RuntimeError("RapidFuzz is required. Install it via: pip install rapidfuzz") from e

# ---------- parameters ----------
FUZZ_THRESHOLD = 85   # lower than Attempt 5 because we add a geo gate
MAX_KM = 25.0         # geographic acceptance radius
EARTH_R_KM = 6371.0
STRATEGY_LABEL = f"hybrid_fuzzy{FUZZ_THRESHOLD}_geo_le{int(MAX_KM)}km_state"

# ---------- helpers ----------
def _ensure_match_columns(df: pd.DataFrame) -> pd.DataFrame:
    if "matched_facility_id" not in df.columns:
        df["matched_facility_id"] = pd.Series([np.nan] * len(df), dtype="object")
    if "match_strategy" not in df.columns:
        df["match_strategy"] = pd.Series([np.nan] * len(df), dtype="object")
    return df

def _progress(a2: pd.DataFrame, label: str):
    total = len(a2)
    matched = a2["matched_facility_id"].notna().sum()
    remaining = total - matched
    pct = (matched / total * 100.0) if total else 0.0
    print(f"[{label}] total rows: {total}")
    print(f"[{label}] matched rows: {matched}")
    print(f"[{label}] remaining unmatched: {remaining}")
    print(f"[{label}] progress: {pct:.2f}%")

def _pick_first_existing(df: pd.DataFrame, options):
    for c in options:
        if c in df.columns:
            return c
    return None

def _haversine_km(lat1, lon1, lat2, lon2):
    lat1 = np.radians(lat1.astype(float)); lon1 = np.radians(lon1.astype(float))
    lat2 = np.radians(lat2.astype(float)); lon2 = np.radians(lon2.astype(float))
    dlat = lat2 - lat1; dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*(np.sin(dlon/2.0)**2)
    return EARTH_R_KM * 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))

def _safe_preview(df, desired_cols, n=10, title=None):
    if title: print(title)
    cols = [c for c in desired_cols if c in df.columns]
    display(df[cols].head(n) if cols else df.head(n))

# ---------- prepare ----------
api2 = _ensure_match_columns(api2)

# required cols
for nm in ["name_norm"]:
    if nm not in api2.columns or nm not in db2.columns:
        raise ValueError(f"Attempt 6 requires column '{nm}' on both api2 and db2.")

# identifier to write back
db_id_candidates = [c for c in ["facility_id", "facility_code", "facility_name"] if c in db2.columns]
if not db_id_candidates:
    raise ValueError("db2 has no identifier column (facility_id / facility_code / facility_name).")
CHOSEN_DB_ID_COL = db_id_candidates[0]

# detect state and coords
api_state_col = _pick_first_existing(api2, ["state_code","state"])
db_state_col  = _pick_first_existing(db2,  ["state_code","state"])
use_state = api_state_col is not None and db_state_col is not None

api_lat_col = _pick_first_existing(api2, ["lat_api","latitude","lat"])
api_lon_col = _pick_first_existing(api2, ["lng_api","longitude","lon","long"])
db_lat_col  = _pick_first_existing(db2,  ["latitude","lat"])
db_lon_col  = _pick_first_existing(db2,  ["longitude","lon","long"])

for name, col in [("api_lat",api_lat_col),("api_lon",api_lon_col),("db_lat",db_lat_col),("db_lon",db_lon_col)]:
    if col is None:
        raise ValueError(f"Attempt 6 requires coordinate column: {name}")

print(f"[Attempt 6] Fuzzy threshold >= {FUZZ_THRESHOLD} and geo gate <= {MAX_KM} km")
print(f"[Attempt 6] Using id '{CHOSEN_DB_ID_COL}'. State gate: {'ON' if use_state else 'OFF'}")

# ---------- candidates (still unmatched + with coords + name) ----------
cand = api2[api2["matched_facility_id"].isna()].copy()
cand = cand[cand["name_norm"].notna() & cand[api_lat_col].notna() & cand[api_lon_col].notna()].copy()
cand = cand.reset_index().rename(columns={"index":"index_api"})

db_ref = db2.copy()
db_ref = db_ref[db_ref["name_norm"].notna() & db_ref[CHOSEN_DB_ID_COL].notna() &
                db_ref[db_lat_col].notna() & db_ref[db_lon_col].notna()].copy()
if use_state:
    db_ref["__state"] = db_ref[db_state_col].astype(str)

accepted_rows_lbl, accepted_ids, accepted_scores, accepted_dists, accepted_best = [], [], [], [], []
rejected_rows_lbl = []

if use_state:
    groups = cand.groupby(cand[api_state_col].astype(str))
else:
    # single group "ALL"
    groups = [("ALL", cand)]

for grp_key, g in groups:
    if isinstance(groups, list):
        g = g  # already DataFrame
    ref = db_ref if not use_state else db_ref[db_ref["__state"] == str(grp_key)]
    if ref.empty:
        continue

    choice_names = ref["name_norm"].astype(str).tolist()
    choice_ids   = ref[CHOSEN_DB_ID_COL].astype(str).tolist()
    lat_db       = ref[db_lat_col].astype(float).to_numpy()
    lon_db       = ref[db_lon_col].astype(float).to_numpy()

    queries = g["name_norm"].astype(str).tolist()
    scores_mat = process.cdist(queries, choice_names, scorer=fuzz.token_set_ratio, workers=0)
    argmax = scores_mat.argmax(axis=1)
    scores = scores_mat.max(axis=1)

    # nearest distance to the same best match candidate
    lat_api = g[api_lat_col].astype(float).to_numpy()
    lon_api = g[api_lon_col].astype(float).to_numpy()
    best_lat = lat_db[argmax]
    best_lon = lon_db[argmax]
    dists = _haversine_km(lat_api, lon_api, best_lat, best_lon)

    sel = (scores >= FUZZ_THRESHOLD) & (dists <= MAX_KM)
    if sel.any():
        accepted_rows_lbl.extend(g["index_api"].to_numpy()[sel].tolist())
        accepted_ids.extend(np.array(choice_ids)[argmax][sel].tolist())
        accepted_scores.extend(scores[sel].astype(float).tolist())
        accepted_dists.extend(dists[sel].astype(float).tolist())
        accepted_best.extend(np.array(choice_names)[argmax][sel].tolist())

    if (~sel).any():
        rejected_rows_lbl.extend(g["index_api"].to_numpy()[~sel].tolist())

# ---------- one-to-one on accepted (keep highest score; if tie, keep closest) ----------
gain = 0
if accepted_rows_lbl:
    acc_df = pd.DataFrame({
        "index_api": accepted_rows_lbl,
        "__chosen_id": accepted_ids,
        "__score": accepted_scores,
        "__dist_km": accepted_dists,
        "__best_name": accepted_best
    })
    acc_df = acc_df.sort_values(["__chosen_id","__score","__dist_km"], ascending=[True, False, True])
    acc_df = acc_df.groupby("__chosen_id", as_index=False).first()
    gain = len(acc_df)

    # write back (label indices are original api2 index values)
    api2.loc[acc_df["index_api"].values, "matched_facility_id"] = acc_df["__chosen_id"].astype(str).values
    api2.loc[acc_df["index_api"].values, "match_strategy"] = STRATEGY_LABEL
    if "matched_distance_km" not in api2.columns:
        api2["matched_distance_km"] = np.nan
    api2.loc[acc_df["index_api"].values, "matched_distance_km"] = acc_df["__dist_km"].astype(float).values

# ---------- summaries ----------
print(f"Hybrid accepted (score >= {FUZZ_THRESHOLD} and dist <= {MAX_KM} km): {gain}")
_progress(api2, label="Attempt 6: hybrid fuzzy+geo")

# ---------- previews ----------
if gain > 0:
    preview = acc_df.merge(
        api2.reset_index().rename(columns={"index":"index_api"})[["index_api","facility_name_api","name_norm"]]
        if "facility_name_api" in api2.columns else
        api2.reset_index().rename(columns={"index":"index_api"})[["index_api","name_norm"]],
        on="index_api", how="left"
    )
    _safe_preview(preview,
                  ["index_api","facility_name_api","name_norm","__best_name","__score","__dist_km","__chosen_id"],
                  n=10, title="Accepted (top 10)")
else:
    print("No accepted rows to preview.")


[Attempt 6] Fuzzy threshold >= 85 and geo gate <= 25.0 km
[Attempt 6] Using id 'facility_code'. State gate: OFF
Hybrid accepted (score >= 85 and dist <= 25.0 km): 0
[Attempt 6: hybrid fuzzy+geo] total rows: 514
[Attempt 6: hybrid fuzzy+geo] matched rows: 398
[Attempt 6: hybrid fuzzy+geo] remaining unmatched: 116
[Attempt 6: hybrid fuzzy+geo] progress: 77.43%
No accepted rows to preview.


In [22]:
# Attempt 7: Alias-based normalisation + fuzzy match (prefer same region/state)
# - Build alias_norm for both datasets (rule-based cleaning and synonym mapping)
# - Fuzzy match remaining unmatched, prefer same region/state when available
# - Write back matched_facility_id + match_strategy, print progress, safe preview

import re
import numpy as np
import pandas as pd

try:
    from rapidfuzz import fuzz, process
except Exception as e:
    raise RuntimeError("RapidFuzz is required. Install via: pip install rapidfuzz") from e

# ---------------- helpers ----------------
def _ensure_match_columns(df: pd.DataFrame) -> pd.DataFrame:
    if "matched_facility_id" not in df.columns:
        df["matched_facility_id"] = pd.Series([np.nan] * len(df), dtype="object")
    if "match_strategy" not in df.columns:
        df["match_strategy"] = pd.Series([np.nan] * len(df), dtype="object")
    return df

def _progress(a2: pd.DataFrame, label: str):
    total = len(a2)
    matched = a2["matched_facility_id"].notna().sum()
    remaining = total - matched
    pct = (matched / total * 100.0) if total else 0.0
    print(f"[{label}] total rows: {total}")
    print(f"[{label}] matched rows: {matched}")
    print(f"[{label}] remaining unmatched: {remaining}")
    print(f"[{label}] progress: {pct:.2f}%")

def _pick_first_existing(df, options):
    for c in options:
        if c in df.columns:
            return c
    return None

def build_alias_norm(s: pd.Series) -> pd.Series:
    """Robust normalisation to collapse descriptive terms and unify common abbreviations."""
    if s is None:
        return pd.Series([], dtype="object")
    x = s.astype(str).str.lower()

    # unify accents/punctuation
    x = (
        x.str.normalize("NFKD")
         .str.encode("ascii", "ignore").str.decode("ascii")
         .str.replace(r"[^\w\s\-&/]", " ", regex=True)
    )

    # common synonym expansions
    repl = [
        (r"\bmt\b", "mount"),
        (r"\bst\b", "saint"),          # st mary -> saint mary
        (r"\bck\b", "creek"),
        (r"\bnth\b", "north"),
        (r"\bsth\b", "south"),
        (r"\bew\b", "east west"),      # keep both tokens if used
        (r"\be\b", "east"),
        (r"\bw\b", "west"),
        (r"\bn\b", "north"),
        (r"\bs\b", "south"),
        (r"\bcs ?m\b", "csm"),
        (r"\bwf\b", "wind farm"),
        (r"\bps\b", "power station"),
    ]
    for pat, rep in repl:
        x = x.str.replace(pat, rep, regex=True)

    # drop descriptive/organisation tokens
    DROP_WORDS = (
        r"\b(power|station|wind|solar|farm|hydro|battery|bess|hub|unit|plant|energy|"
        r"ps|wf|csm|company|co|pty|ltd|limited|plc|trust|holdings?|project|substation|"
        r"facility|site|assets?)\b"
    )
    x = x.str.replace(DROP_WORDS, " ", regex=True)

    # unify connectors
    x = x.str.replace(r"[/&\-]", " ", regex=True)

    # remove numbers that are standalone (keep if part of name? choose to drop)
    x = x.str.replace(r"\b\d+\b", " ", regex=True)

    # collapse spaces and trim
    x = x.str.replace(r"\s+", " ", regex=True).str.strip()

    return x

def _safe_preview(df, desired_cols, n=10, title=None):
    if title: print(title)
    if df is None or len(df) == 0:
        print("(empty)")
        return
    cols = [c for c in desired_cols if c in df.columns]
    display(df[cols].head(n) if cols else df.head(n))

# ---------------- setup ----------------
api2 = _ensure_match_columns(api2)

# guards
if "name_norm" not in api2.columns or "name_norm" not in db2.columns:
    raise ValueError("Attempt 7 requires 'name_norm' on both api2 and db2.")

# identifier to write to matched_facility_id
db_id_candidates = [c for c in ["facility_id", "facility_code", "facility_name"] if c in db2.columns]
if not db_id_candidates:
    raise ValueError("db2 has no identifier column (facility_id / facility_code / facility_name).")
CHOSEN_DB_ID_COL = db_id_candidates[0]

api_region_col = _pick_first_existing(api2, ["region_api", "region"])
db_region_col  = _pick_first_existing(db2,  ["region_db", "region"])
api_state_col  = _pick_first_existing(api2, ["state_code", "state"])
db_state_col   = _pick_first_existing(db2,  ["state_code", "state"])
use_region = api_region_col is not None and db_region_col is not None
use_state  = api_state_col is not None and db_state_col is not None

# parameters
FUZZ_THRESHOLD = 88
SCORER = fuzz.token_set_ratio
STRATEGY_LABEL = f"alias_fuzzy_tokenset_{FUZZ_THRESHOLD}"

print(f"[Attempt 7] Identifier: '{CHOSEN_DB_ID_COL}'. Threshold ≥ {FUZZ_THRESHOLD}. "
      f"Prefer region: {use_region}, state: {use_state}")

# ---------------- build alias_norm ----------------
if "alias_norm" not in api2.columns:
    api2["alias_norm"] = build_alias_norm(api2["name_norm"])
if "alias_norm" not in db2.columns:
    db2["alias_norm"] = build_alias_norm(db2["name_norm"])

# candidates: still unmatched and have alias_norm
cand = api2[api2["matched_facility_id"].isna() & api2["alias_norm"].notna() & (api2["alias_norm"].str.len() > 0)].copy()
cand = cand.reset_index().rename(columns={"index": "index_api"})

# db ref
db_ref = db2[db2["alias_norm"].notna() & (db2["alias_norm"].str.len() > 0) & db2[CHOSEN_DB_ID_COL].notna()].copy()
db_ref["__chosen_id"] = db_ref[CHOSEN_DB_ID_COL].astype(str)
if use_region: db_ref["__region"] = db_ref[db_region_col].astype(str)
if use_state:  db_ref["__state"] = db_ref[db_state_col].astype(str)

accepted_rows, accepted_ids, accepted_scores, accepted_best = [], [], [], []
rejected_rows = []

def _match_block(block_df: pd.DataFrame, ref_df: pd.DataFrame):
    """Run fuzzy matching of alias_norm for a block against reference."""
    if ref_df.empty or block_df.empty:
        return [], [], [], [], []
    queries = block_df["alias_norm"].astype(str).tolist()
    choice_names = ref_df["alias_norm"].astype(str).tolist()
    choice_ids   = ref_df["__chosen_id"].tolist()
    scores_mat = process.cdist(queries, choice_names, scorer=SCORER, workers=0)
    argmax = scores_mat.argmax(axis=1)
    scores = scores_mat.max(axis=1)

    sel = scores >= FUZZ_THRESHOLD
    rows  = block_df["index_api"].to_numpy()[sel].tolist()
    ids   = np.array(choice_ids)[argmax][sel].tolist()
    sc    = scores[sel].astype(float).tolist()
    bestn = np.array(choice_names)[argmax][sel].tolist()

    rej_rows = block_df["index_api"].to_numpy()[~sel].tolist()
    return rows, ids, sc, bestn, rej_rows

# Priority 1: within same region (if available)
if use_region:
    for rg, g in cand.groupby(cand[api_region_col].astype(str)):
        ref = db_ref[db_ref["__region"] == str(rg)]
        rows, ids, sc, bestn, rej = _match_block(g, ref)
        accepted_rows += rows; accepted_ids += ids; accepted_scores += sc; accepted_best += bestn
        rejected_rows += rej

# Priority 2: fallback within same state (if available and not already matched here)
if use_state:
    remain = cand[~cand["index_api"].isin(accepted_rows)].copy()
    for st, g in remain.groupby(remain[api_state_col].astype(str)):
        ref = db_ref[db_ref["__state"] == str(st)]
        rows, ids, sc, bestn, rej = _match_block(g, ref)
        # avoid duplicating acceptance for the same id (keep highest score)
        accepted_rows += rows; accepted_ids += ids; accepted_scores += sc; accepted_best += bestn
        rejected_rows += rej

# Priority 3: global fallback (only for those still not matched)
remain = cand[~cand["index_api"].isin(accepted_rows)].copy()
rows, ids, sc, bestn, rej = _match_block(remain, db_ref)
accepted_rows += rows; accepted_ids += ids; accepted_scores += sc; accepted_best += bestn
rejected_rows += rej

# One-to-one resolution per chosen_id: keep highest score
gain = 0
if accepted_rows:
    acc_df = pd.DataFrame({
        "index_api": accepted_rows,
        "__chosen_id": accepted_ids,
        "__score": accepted_scores,
        "__best_name": accepted_best
    })
    acc_df = acc_df.sort_values(["__chosen_id","__score"], ascending=[True, False]).groupby("__chosen_id", as_index=False).first()
    gain = len(acc_df)

    api2.loc[acc_df["index_api"].values, "matched_facility_id"] = acc_df["__chosen_id"].astype(str).values
    api2.loc[acc_df["index_api"].values, "match_strategy"] = STRATEGY_LABEL

print(f"Alias-fuzzy accepted (>= {FUZZ_THRESHOLD}): {gain}")
_progress(api2, label="Attempt 7: alias fuzzy")

# previews
if gain > 0:
    preview = acc_df.merge(
        api2.reset_index().rename(columns={"index":"index_api"})[
            ["index_api","facility_name_api","name_norm","alias_norm"]
        ] if "facility_name_api" in api2.columns else
        api2.reset_index().rename(columns={"index":"index_api"})[
            ["index_api","name_norm","alias_norm"]
        ],
        on="index_api", how="left"
    )
    _safe_preview(preview,
                  ["index_api","facility_name_api","name_norm","alias_norm","__best_name","__score","__chosen_id"],
                  n=10, title="Accepted (top 10) — Attempt 7")
else:
    print("No accepted rows to preview.")

[Attempt 7] Identifier: 'facility_code'. Threshold ≥ 88. Prefer region: True, state: False
Alias-fuzzy accepted (>= 88): 19
[Attempt 7: alias fuzzy] total rows: 514
[Attempt 7: alias fuzzy] matched rows: 417
[Attempt 7: alias fuzzy] remaining unmatched: 97
[Attempt 7: alias fuzzy] progress: 81.13%
Accepted (top 10) — Attempt 7


,index_api,name_norm,alias_norm,__best_name,__score,__chosen_id
0,131,eildon,eildon,eildon,100.000000,12
1,235,lake bonney 3,lake bonney,lake bonney stage,100.000000,218
2,495,woodlawn,woodlawn,woodlawn,100.000000,219
3,254,loy yang a,loy yang a,loy yang and mine,88.888885,23
4,25,bell bay,bell bay,bell bay thermal,100.000000,369
5,224,kidston,kidston,kidston one qld,100.000000,493
6,193,hornsdale 2,hornsdale,hornsdale reserve,100.000000,506
7,146,gannawarra,gannawarra,gannawarra,100.000000,609
8,316,oakey 2,oakey,oakey,100.000000,616
9,156,glenrowan west,glenrowan west,glenrowan west,100.000000,645


In [23]:
# Attempt 8 (robust): Geo expansion by facility type (larger radius for Wind/Solar)
# - Fix preview to avoid KeyError when optional columns are missing
# - Keep the same matching logic; print progress and safe previews

import pandas as pd
import numpy as np

BASE_KM = 15.0
EXPANDED_KM = 50.0
EARTH_R_KM = 6371.0
STRATEGY_LABEL = "geo_expanded_typeaware"

def _ensure_match_columns(df: pd.DataFrame) -> pd.DataFrame:
    if "matched_facility_id" not in df.columns:
        df["matched_facility_id"] = np.nan
    if "match_strategy" not in df.columns:
        df["match_strategy"] = np.nan
    return df

def _progress(a2: pd.DataFrame, label: str):
    total = len(a2)
    matched = a2["matched_facility_id"].notna().sum()
    remaining = total - matched
    pct = (matched / total * 100.0) if total else 0.0
    print(f"[{label}] total rows: {total}")
    print(f"[{label}] matched rows: {matched}")
    print(f"[{label}] remaining unmatched: {remaining}")
    print(f"[{label}] progress: {pct:.2f}%")

def _pick_first_existing(df, options):
    for c in options:
        if c in df.columns:
            return c
    return None

def _haversine_km(lat1, lon1, lat2, lon2):
    lat1 = np.radians(lat1.astype(float)); lon1 = np.radians(lon1.astype(float))
    lat2 = np.radians(lat2.astype(float)); lon2 = np.radians(lon2.astype(float))
    dlat = lat2 - lat1; dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*(np.sin(dlon/2.0)**2)
    return EARTH_R_KM * 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))

def _safe_preview(df, desired_cols, n=10, title=None):
    if title: print(title)
    if df is None or len(df) == 0:
        print("(empty)")
        return
    cols = [c for c in desired_cols if c in df.columns]
    display(df[cols].head(n) if cols else df.head(n))

# ---------- setup ----------
api2 = _ensure_match_columns(api2)

api_lat_col = _pick_first_existing(api2, ["lat_api","latitude","lat"])
api_lon_col = _pick_first_existing(api2, ["lng_api","longitude","lon","long"])
db_lat_col  = _pick_first_existing(db2,  ["latitude","lat"])
db_lon_col  = _pick_first_existing(db2,  ["longitude","lon","long"])
api_region_col = _pick_first_existing(api2, ["region_api","region"])
db_region_col  = _pick_first_existing(db2,  ["region_db","region"])
api_state_col  = _pick_first_existing(api2, ["state_code","state"])
db_state_col   = _pick_first_existing(db2,  ["state_code","state"])

id_candidates = [c for c in ["facility_id","facility_code","facility_name"] if c in db2.columns]
if not id_candidates:
    raise ValueError("db2 has no identifier column (facility_id / facility_code / facility_name).")
CHOSEN_DB_ID_COL = id_candidates[0]

print(f"[Attempt 8] Using id: {CHOSEN_DB_ID_COL}. Base radius = {BASE_KM} km, expanded = {EXPANDED_KM} km.")

# Candidates: still unmatched + have coords
cand = api2[(api2["matched_facility_id"].isna()) &
            api2[api_lat_col].notna() & api2[api_lon_col].notna()].copy()

# Ensure we have a fuel_type column to test (fallback to empty string)
if "fuel_type" not in cand.columns:
    cand["fuel_type"] = ""

db_ref = db2[(db2[CHOSEN_DB_ID_COL].notna()) &
             db2[db_lat_col].notna() & db2[db_lon_col].notna()].copy()

accepted_rows, accepted_ids, accepted_dists = [], [], []

for idx, row in cand.iterrows():
    lat1, lon1 = float(row[api_lat_col]), float(row[api_lon_col])
    ftype = str(row["fuel_type"]).lower()
    radius = EXPANDED_KM if any(k in ftype for k in ["wind", "solar"]) else BASE_KM

    # Prefer region match, otherwise fall back to state if available
    ref = db_ref
    if api_region_col and db_region_col and pd.notna(row.get(api_region_col, np.nan)):
        ref = ref[ref[db_region_col] == row[api_region_col]]
    elif api_state_col and db_state_col and pd.notna(row.get(api_state_col, np.nan)):
        ref = ref[ref[db_state_col] == row[api_state_col]]

    if ref.empty:
        continue

    dists = _haversine_km(
        np.full(len(ref), lat1),
        np.full(len(ref), lon1),
        ref[db_lat_col].to_numpy(),
        ref[db_lon_col].to_numpy()
    )
    j = int(np.argmin(dists))
    dist_best = float(dists[j])

    if dist_best <= radius:
        accepted_rows.append(idx)
        accepted_ids.append(str(ref.iloc[j][CHOSEN_DB_ID_COL]))
        accepted_dists.append(dist_best)

# Write back
if accepted_rows:
    api2.loc[accepted_rows, "matched_facility_id"] = accepted_ids
    api2.loc[accepted_rows, "match_strategy"] = STRATEGY_LABEL
    if "matched_distance_km" not in api2.columns:
        api2["matched_distance_km"] = np.nan
    api2.loc[accepted_rows, "matched_distance_km"] = accepted_dists
    print(f"Accepted by geo-expansion: {len(accepted_rows)}")
else:
    print("No additional matches found via geo-expansion.")

# Progress + robust previews (select columns inside the function)
_progress(api2, "Attempt 8: geo expansion by fuel type")
preview_df = api2.loc[accepted_rows].copy() if accepted_rows else pd.DataFrame()
_safe_preview(
    preview_df,
    desired_cols=[api_region_col, "fuel_type", "matched_facility_id", "matched_distance_km", "match_strategy"],
    n=10,
    title="Accepted (top 10)"
)


[Attempt 8] Using id: facility_code. Base radius = 15.0 km, expanded = 50.0 km.
Accepted by geo-expansion: 1
[Attempt 8: geo expansion by fuel type] total rows: 514
[Attempt 8: geo expansion by fuel type] matched rows: 418
[Attempt 8: geo expansion by fuel type] remaining unmatched: 96
[Attempt 8: geo expansion by fuel type] progress: 81.32%
Accepted (top 10)


,region_api,matched_facility_id,matched_distance_km,match_strategy
383,NSW1,281,11.349188,geo_expanded_typeaware


In [24]:
# Attempt 9: Relaxed fuzzy within state + same fuel_category + geo gate (robust numpy alias)
# Uses local numpy alias ⁠ _N ⁠ inside haversine to avoid collisions with any variable named ⁠ np ⁠.

import numpy as _N
import pandas as pd
import numpy as np  # keep existing alias but haversine will NOT rely on it
from rapidfuzz import fuzz, process

# ---- parameters ----
FUZZ_THRESHOLD = 80
MAX_KM = 35.0
EARTH_R_KM = 6371.0
STRATEGY_LABEL = f"relaxed_fuzzy_state_fuel_geo_{FUZZ_THRESHOLD}_{int(MAX_KM)}km"

def _ensure_match_columns(df: pd.DataFrame) -> pd.DataFrame:
    if "matched_facility_id" not in df.columns:
        df["matched_facility_id"] = pd.Series([_N.nan] * len(df), dtype="object")
    if "match_strategy" not in df.columns:
        df["match_strategy"] = pd.Series([_N.nan] * len(df), dtype="object")
    return df

def _progress(a2: pd.DataFrame, label: str):
    total = len(a2)
    matched = a2["matched_facility_id"].notna().sum()
    remaining = total - matched
    pct = (matched / total * 100.0) if total else 0.0
    print(f"[{label}] total rows: {total}")
    print(f"[{label}] matched rows: {matched}")
    print(f"[{label}] remaining unmatched: {remaining}")
    print(f"[{label}] progress: {pct:.2f}%")

def _pick_first_existing(df: pd.DataFrame, options):
    for c in options:
        if c in df.columns:
            return c
    return None

def _haversine_km(lat1, lon1, lat2, lon2):
    """Use local numpy alias to avoid np-collision in user namespace."""
    lat1 = _N.radians(_N.asarray(lat1, dtype=float))
    lon1 = _N.radians(_N.asarray(lon1, dtype=float))
    lat2 = _N.radians(_N.asarray(lat2, dtype=float))
    lon2 = _N.radians(_N.asarray(lon2, dtype=float))
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = _N.sin(dlat/2.0)*2 + _N.cos(lat1) * _N.cos(lat2) * (_N.sin(dlon/2.0)*2)
    return EARTH_R_KM * 2.0 * _N.arctan2(_N.sqrt(a), _N.sqrt(1.0 - a))

def _safe_preview(df, desired_cols, n=10, title=None):
    if title: print(title)
    if df is None or len(df) == 0:
        print("(empty)")
        return
    cols = [c for c in desired_cols if c in df.columns]
    display(df[cols].head(n) if cols else df.head(n))

# ---- setup ----
api2 = _ensure_match_columns(api2)
if "name_norm" not in api2.columns or "name_norm" not in db2.columns:
    raise ValueError("Attempt 9r requires 'name_norm' on both api2 and db2.")

db_id_candidates = [c for c in ["facility_id","facility_code","facility_name"] if c in db2.columns]
if not db_id_candidates:
    raise ValueError("db2 has no identifier column (facility_id / facility_code / facility_name).")
CHOSEN_DB_ID_COL = db_id_candidates[0]

api_state_col  = _pick_first_existing(api2, ["state_code","state"])
db_state_col   = _pick_first_existing(db2,  ["state_code","state"])
use_state_gate = api_state_col is not None and db_state_col is not None

api_lat_col = _pick_first_existing(api2, ["lat_api","latitude","lat"])
api_lon_col = _pick_first_existing(api2, ["lng_api","longitude","lon","long"])
db_lat_col  = _pick_first_existing(db2,  ["latitude","lat"])
db_lon_col  = _pick_first_existing(db2,  ["longitude","lon","long"])
for nm, col in [("api_lat",api_lat_col),("api_lon",api_lon_col),("db_lat",db_lat_col),("db_lon",db_lon_col)]:
    if col is None:
        raise ValueError(f"Attempt 9r needs coordinate column: {nm}")

api_fuel_cat = _pick_first_existing(api2, ["fuel_category","fuel_cat","fuel_group"])
db_fuel_cat  = _pick_first_existing(db2,  ["fuel_category","fuel_cat","fuel_group"])
use_fuel_gate = api_fuel_cat is not None and db_fuel_cat is not None

print(f"[Attempt 9r] ID='{CHOSEN_DB_ID_COL}', fuzzy≥{FUZZ_THRESHOLD}, geo≤{MAX_KM} km, "
      f"state gate: {'ON' if use_state_gate else 'OFF'}, fuel gate: {'ON' if use_fuel_gate else 'OFF'}")

# ---- candidates ----
cand = api2[api2["matched_facility_id"].isna()].copy()
cand = cand[cand["name_norm"].notna() & cand[api_lat_col].notna() & cand[api_lon_col].notna()].copy()
cand = cand.reset_index().rename(columns={"index":"index_api"})

db_ref = db2.copy()
db_ref = db_ref[db_ref["name_norm"].notna() & db_ref[CHOSEN_DB_ID_COL].notna() &
                db_ref[db_lat_col].notna() & db_ref[db_lon_col].notna()].copy()
if use_state_gate: db_ref["__state"] = db_ref[db_state_col].astype(str)
if use_fuel_gate:  db_ref["__fuel"]  = db_ref[db_fuel_cat].astype(str)
db_ref["__chosen_id"] = db_ref[CHOSEN_DB_ID_COL].astype(str)

accepted_rows, accepted_ids, accepted_scores, accepted_dists, accepted_best = [], [], [], [], []
rejected_rows = []

scorer = fuzz.token_sort_ratio

if use_state_gate:
    groups = cand.groupby(cand[api_state_col].astype(str))
else:
    groups = [("ALL", cand)]

for grp_key, g in groups:
    ref_state = db_ref if not use_state_gate else db_ref[db_ref["__state"] == str(grp_key)]
    if ref_state.empty:
        continue

    if use_fuel_gate:
        for fuel, g2 in g.groupby(g[api_fuel_cat].astype(str)):
            ref = ref_state[ref_state["__fuel"] == str(fuel)]
            if ref.empty:
                continue

            qnames = g2["name_norm"].astype(str).tolist()
            choice_names = ref["name_norm"].astype(str).tolist()
            choice_ids   = ref["__chosen_id"].tolist()
            lat_db, lon_db = ref[db_lat_col].astype(float).to_numpy(), ref[db_lon_col].astype(float).to_numpy()

            scores_mat = process.cdist(qnames, choice_names, scorer=scorer, workers=0)
            argmax = scores_mat.argmax(axis=1)
            scores = scores_mat.max(axis=1)

            lat_api = g2[api_lat_col].astype(float).to_numpy()
            lon_api = g2[api_lon_col].astype(float).to_numpy()
            best_lat = lat_db[argmax]; best_lon = lon_db[argmax]
            dists = _haversine_km(lat_api, lon_api, best_lat, best_lon)

            sel = (scores >= FUZZ_THRESHOLD) & (dists <= MAX_KM)
            if sel.any():
                accepted_rows.extend(g2["index_api"].to_numpy()[sel].tolist())
                accepted_ids.extend(_N.array(choice_ids)[argmax][sel].tolist())
                accepted_scores.extend(scores[sel].astype(float).tolist())
                accepted_dists.extend(dists[sel].astype(float).tolist())
                accepted_best.extend(_N.array(choice_names)[argmax][sel].tolist())

            rejected_rows.extend(g2["index_api"].to_numpy()[~sel].tolist())
    else:
        qnames = g["name_norm"].astype(str).tolist()
        choice_names = ref_state["name_norm"].astype(str).tolist()
        choice_ids   = ref_state["__chosen_id"].tolist()
        lat_db, lon_db = ref_state[db_lat_col].astype(float).to_numpy(), ref_state[db_lon_col].astype(float).to_numpy()

        scores_mat = process.cdist(qnames, choice_names, scorer=scorer, workers=0)
        argmax = scores_mat.argmax(axis=1)
        scores = scores_mat.max(axis=1)

        lat_api = g[api_lat_col].astype(float).to_numpy()
        lon_api = g[api_lon_col].astype(float).to_numpy()
        best_lat = lat_db[argmax]; best_lon = lon_db[argmax]
        dists = _haversine_km(lat_api, lon_api, best_lat, best_lon)

        sel = (scores >= FUZZ_THRESHOLD) & (dists <= MAX_KM)
        if sel.any():
            accepted_rows.extend(g["index_api"].to_numpy()[sel].tolist())
            accepted_ids.extend(_N.array(choice_ids)[argmax][sel].tolist())
            accepted_scores.extend(scores[sel].astype(float).tolist())
            accepted_dists.extend(dists[sel].astype(float).tolist())
            accepted_best.extend(_N.array(choice_names)[argmax][sel].tolist())

        rejected_rows.extend(g["index_api"].to_numpy()[~sel].tolist())

# one-to-one resolution
gain = 0
if accepted_rows:
    acc_df = pd.DataFrame({
        "index_api": accepted_rows,
        "__chosen_id": accepted_ids,
        "__score": accepted_scores,
        "__dist_km": accepted_dists,
        "__best_name": accepted_best
    })
    acc_df = acc_df.sort_values(["__chosen_id","__score","__dist_km"], ascending=[True, False, True])
    acc_df = acc_df.groupby("__chosen_id", as_index=False).first()
    gain = len(acc_df)

    api2.loc[acc_df["index_api"].values, "matched_facility_id"] = acc_df["__chosen_id"].astype(str).values
    api2.loc[acc_df["index_api"].values, "match_strategy"] = STRATEGY_LABEL
    if "matched_distance_km" not in api2.columns:
        api2["matched_distance_km"] = _N.nan
    api2.loc[acc_df["index_api"].values, "matched_distance_km"] = acc_df["__dist_km"].astype(float).values

print(f"Relaxed-fuzzy accepted (retry): {gain}")
_progress(api2, label="Attempt 9r: relaxed fuzzy + gates")

# preview (robust)
if gain > 0:
    preview = acc_df.merge(
        api2.reset_index().rename(columns={"index":"index_api"})[
            [c for c in ["index_api","facility_name_api","name_norm", api_state_col, api_fuel_cat] if c]
        ],
        on="index_api", how="left"
    )
    _safe_preview(preview,
                  desired_cols=[c for c in ["index_api","facility_name_api","name_norm","__best_name",
                                            "__score","__dist_km","__chosen_id"] if True],
                  n=10, title="Accepted (top 10) — Attempt 9r")
else:
    print("No accepted rows to preview.")

[Attempt 9r] ID='facility_code', fuzzy≥80, geo≤35.0 km, state gate: OFF, fuel gate: OFF
Relaxed-fuzzy accepted (retry): 0
[Attempt 9r: relaxed fuzzy + gates] total rows: 514
[Attempt 9r: relaxed fuzzy + gates] matched rows: 418
[Attempt 9r: relaxed fuzzy + gates] remaining unmatched: 96
[Attempt 9r: relaxed fuzzy + gates] progress: 81.32%
No accepted rows to preview.


C:\Users\Rengga\AppData\Local\Temp\ipykernel_19628\4138642062.py:47: RuntimeWarning: invalid value encountered in sqrt
  return EARTH_R_KM * 2.0 * _N.arctan2(_N.sqrt(a), _N.sqrt(1.0 - a))


In [25]:
# This does not change matching results, only re-renders the preview

# guard: acc_df is created in Attempt 10 when gain > 0
if 'acc_df' in globals() and isinstance(acc_df, pd.DataFrame) and not acc_df.empty:
    api2_with_idx = api2.reset_index().rename(columns={"index":"index_api"})
    # choose only columns that actually exist in api2
    preview_cols = [c for c in ["index_api", "facility_name_api", "facility_name", "name_norm", api_state_col]
                    if c and c in api2_with_idx.columns]

    preview = acc_df.merge(
        api2_with_idx[preview_cols],
        on="index_api",
        how="left"
    )

    # desired columns (with safe membership check)
    desired_cols = [c for c in ["index_api", "facility_name_api", "facility_name",
                                "name_norm", "__overlap", "__dist_km", "__chosen_id"]
                    if c in preview.columns]

    print("Accepted (top 10) — Attempt 10")
    display(preview[desired_cols].head(10) if desired_cols else preview.head(10))
else:
    print("No accepted rows to preview (acc_df missing or empty).")

Accepted (top 10) — Attempt 10


,index_api,facility_name,name_norm,__chosen_id
0,131,Eildon,eildon,12
1,235,Lake Bonney 3,lake bonney 3,218
2,495,Woodlawn,woodlawn,219
3,254,Loy Yang A,loy yang a,23
4,25,Bell Bay,bell bay,369
5,224,Kidston,kidston,493
6,193,Hornsdale 2,hornsdale 2,506
7,146,Gannawarra,gannawarra,609
8,316,Oakey 2,oakey 2,616
9,156,Glenrowan West,glenrowan west,645


In [26]:
# Attempt 10 (clean retry): Token-overlap (Jaccard) + geographic gate (<= 35 km), prefer same state
# Runs only on currently-unmatched rows, writes matched_facility_id and match_strategy, and prints progress.

import numpy as np
import pandas as pd
import re

# ----- parameters -----
MAX_KM = 35.0
EARTH_R_KM = 6371.0
STRATEGY_LABEL = f"token_overlap_geo_{int(MAX_KM)}km"

# ----- helpers -----
def _ensure_match_columns(df: pd.DataFrame) -> pd.DataFrame:
    if "matched_facility_id" not in df.columns:
        df["matched_facility_id"] = pd.Series([np.nan] * len(df), dtype="object")
    if "match_strategy" not in df.columns:
        df["match_strategy"] = pd.Series([np.nan] * len(df), dtype="object")
    return df

def _progress(a2: pd.DataFrame, label: str):
    total = len(a2)
    matched = a2["matched_facility_id"].notna().sum()
    remaining = total - matched
    pct = (matched / total * 100.0) if total else 0.0
    print(f"[{label}] total rows: {total}")
    print(f"[{label}] matched rows: {matched}")
    print(f"[{label}] remaining unmatched: {remaining}")
    print(f"[{label}] progress: {pct:.2f}%")

def _pick_first_existing(df, options):
    for c in options:
        if c in df.columns:
            return c
    return None

def _haversine_km(lat1, lon1, lat2, lon2):
    lat1 = np.radians(np.asarray(lat1, dtype="float64"))
    lon1 = np.radians(np.asarray(lon1, dtype="float64"))
    lat2 = np.radians(np.asarray(lat2, dtype="float64"))
    lon2 = np.radians(np.asarray(lon2, dtype="float64"))
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)*2 + np.cos(lat1) * np.cos(lat2) * (np.sin(dlon/2.0)*2)
    a = np.clip(a, 0.0, 1.0)  # numerical safety
    return EARTH_R_KM * 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1.0 - a))

TOKEN_RX = re.compile(r"[a-z]+")

def tokens(s: str):
    if not isinstance(s, str):
        return set()
    return set(TOKEN_RX.findall(s.lower()))

def jaccard(a: set, b: set) -> float:
    if not a or not b:
        return 0.0
    inter = len(a & b)
    union = len(a | b)
    return inter / union if union else 0.0

def _safe_preview(df, desired_cols, n=10, title=None):
    if title: print(title)
    if df is None or len(df) == 0:
        print("(empty)")
        return
    cols = [c for c in desired_cols if c in df.columns]
    display(df[cols].head(n) if cols else df.head(n))

# ----- setup columns -----
api2 = _ensure_match_columns(api2)

# name field must exist on both
if "name_norm" not in api2.columns or "name_norm" not in db2.columns:
    raise ValueError("Attempt 10 requires 'name_norm' on both api2 and db2.")

# choose identifier column from db2 to write back
id_candidates = [c for c in ["facility_id", "facility_code", "facility_name"] if c in db2.columns]
if not id_candidates:
    raise ValueError("db2 needs an identifier column (facility_id / facility_code / facility_name).")
CHOSEN_DB_ID_COL = id_candidates[0]

# coordinates
api_lat_col = _pick_first_existing(api2, ["lat_api", "latitude", "lat"])
api_lon_col = _pick_first_existing(api2, ["lng_api", "longitude", "lon", "long"])
db_lat_col  = _pick_first_existing(db2,  ["latitude", "lat"])
db_lon_col  = _pick_first_existing(db2,  ["longitude", "lon", "long"])
for nm, col in [("api_lat", api_lat_col), ("api_lon", api_lon_col), ("db_lat", db_lat_col), ("db_lon", db_lon_col)]:
    if col is None:
        raise ValueError(f"Attempt 10 needs coordinate column: {nm}")

# optional state gate
api_state_col = _pick_first_existing(api2, ["state_code", "state"])
db_state_col  = _pick_first_existing(db2,  ["state_code", "state"])
use_state = api_state_col is not None and db_state_col is not None

# ----- build token caches -----
api2["_tok"] = api2["name_norm"].apply(tokens)
db2["_tok"]  = db2["name_norm"].apply(tokens)

# ----- candidate subsets -----
cand = api2[api2["matched_facility_id"].isna()].copy()
cand = cand[cand["_tok"].apply(len) > 0]
cand = cand[cand[api_lat_col].notna() & cand[api_lon_col].notna()].copy()
cand = cand.reset_index().rename(columns={"index": "index_api"})

db_ref = db2.copy()
db_ref = db_ref[db_ref["_tok"].apply(len) > 0]
db_ref = db_ref[db_ref[CHOSEN_DB_ID_COL].notna() & db_ref[db_lat_col].notna() & db_ref[db_lon_col].notna()].copy()
db_ref["__chosen_id"] = db_ref[CHOSEN_DB_ID_COL].astype(str)
if use_state:
    db_ref["__state"] = db_ref[db_state_col].astype(str)

# ----- main loop -----
accepted = []

for _, row in cand.iterrows():
    pool = db_ref
    # prefer same state if available
    if use_state and pd.notna(row.get(api_state_col, np.nan)):
        pool = pool[pool["__state"] == str(row[api_state_col])]
        if pool.empty:
            pool = db_ref  # fallback to global pool

    if pool.empty:
        continue

    a_tok = row["_tok"]
    # compute jaccard to all pool rows
    scores = np.array([jaccard(a_tok, b) for b in pool["_tok"].tolist()], dtype="float64")
    if scores.size == 0 or float(np.nanmax(scores)) <= 0.0:
        continue

    k = int(np.nanargmax(scores))
    best = pool.iloc[k]
    best_score = float(scores[k])

    # distance gate
    d_km = float(_haversine_km(
        np.array([row[api_lat_col]]), np.array([row[api_lon_col]]),
        np.array([best[db_lat_col]]),  np.array([best[db_lon_col]])
    )[0])

    if d_km <= MAX_KM and best_score > 0.0:
        accepted.append((
            row["index_api"], str(best["__chosen_id"]), best_score, d_km
        ))

# ----- write back & report -----
gain = 0
if accepted:
    acc_df = pd.DataFrame(accepted, columns=["index_api", "__chosen_id", "__overlap", "__dist_km"])
    # one-to-one on id: highest overlap then nearest
    acc_df = acc_df.sort_values(["__chosen_id", "__overlap", "__dist_km"], ascending=[True, False, True])
    acc_df = acc_df.groupby("__chosen_id", as_index=False).first()
    gain = len(acc_df)

    api2.loc[acc_df["index_api"].values, "matched_facility_id"] = acc_df["__chosen_id"].astype(str).values
    api2.loc[acc_df["index_api"].values, "match_strategy"] = STRATEGY_LABEL
    if "matched_distance_km" not in api2.columns:
        api2["matched_distance_km"] = np.nan
    api2.loc[acc_df["index_api"].values, "matched_distance_km"] = acc_df["__dist_km"].astype(float).values

print(f"Token-overlap accepted in this attempt: {gain}")
_progress(api2, label="Attempt 10: token-overlap + geo")

# ----- safe preview -----
if gain > 0:
    api2_with_idx = api2.reset_index().rename(columns={"index": "index_api"})
    preview_cols = [c for c in ["index_api", "facility_name_api", "facility_name", "name_norm", api_state_col] if c in api2_with_idx.columns]
    preview = acc_df.merge(api2_with_idx[preview_cols], on="index_api", how="left")
    desired_cols = [c for c in ["index_api", "facility_name_api", "facility_name", "name_norm", "__overlap", "__dist_km", "__chosen_id"] if c in preview.columns]
    _safe_preview(preview, desired_cols, n=10, title="Accepted (top 10) — Attempt 10")
else:
    print("No accepted rows to preview.")

Token-overlap accepted in this attempt: 10
[Attempt 10: token-overlap + geo] total rows: 514
[Attempt 10: token-overlap + geo] matched rows: 428
[Attempt 10: token-overlap + geo] remaining unmatched: 86
[Attempt 10: token-overlap + geo] progress: 83.27%
Accepted (top 10) — Attempt 10


,index_api,facility_name,name_norm,__overlap,__dist_km,__chosen_id
0,227,Kogan Creek,kogan creek,0.333333,0.0,138
1,478,Whitwood Road,whitwood road,0.250000,0.0,141
2,402,Swanbank E,swanbank e,0.250000,0.0,240
3,299,Mugga Lane,mugga lane,0.250000,0.0,253
4,341,Quorn Park,quorn park,0.250000,0.0,260
5,485,Windy Hill,windy hill,0.200000,0.0,28
6,458,Waratah Super Battery,waratah super battery,0.200000,0.0,698
7,327,Phillip Island,phillip island,0.333333,0.0,755
8,203,Invicta Sugar Mill,invicta sugar mill,0.333333,0.0,787
9,138,Eraring 2,eraring 2,0.250000,0.0,799


### <b> <span style="color:pink">1.2 Get All Power Generated and CO2 Emissions per Facility</span></b>


Facility data will return total data instead of facility specific data if we do not specify the facility code in tha API call. Hence, we need to pass the facility code we get from previous API call to this endpoint.

In [27]:
# get all facility code
FACILITY_LIST = facilities_df["code"].tolist()
print(f"Total facilities: {len(FACILITY_LIST)}")

Total facilities: 514


However, since the parameter only accept 30 max characters and we have more than 500 facilities, passing all facility code at once will result in error. To get around that, we will use the batching strategy.

In [28]:
OUT_DIR = "./DATA/EXTRACTED"
os.makedirs(OUT_DIR, exist_ok=True)
ENDPOINT = "data/facilities/NEM"
batch_size = 5

# function chunk the facility code list into batches
def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n], i, min(i+n, len(lst))  

# batch retrieval
def batch_retrieval(batch_size:int):
    for batch, start, end in chunk_list(FACILITY_LIST, batch_size):
        batch_id = math.ceil(end/batch_size)
        cache_path = os.path.join(OUT_DIR, f"batch_{start+1:04d}_{end:04d}.json")

        if os.path.exists(cache_path):
            print(f"Batch {batch_id}: {start+1}-{end} already cached.")
            continue

        print(f"Fetching batch {batch_id}: facilities {start+1}–{end} ({batch})")

        # params
        params = {
            'network_code': 'NEM',
            'metrics': {'power', 'emissions'},
            'interval': '5m',
            "date_start": "2025-10-01",
            "date_end": "2025-10-08",
            "facility_code": {f for f in batch},
        }

        try:
            r = requests.get(f"{BASE_URL}{ENDPOINT}", headers=HEADERS, params=params, timeout=90)
            if r.status_code == 200:
                payload = r.json()
                with open(cache_path, "w") as f:
                    json.dump(payload, f, indent=2)
                print(f"Saved {cache_path}")
            else:
                print(f"HTTP {r.status_code}: {r.text[:150]}")
        except Exception as e:
            print(f"Batch {batch_id}: {e}")

        time.sleep(0.3)

# function to flatten the payload
def flatten(payload):
    rows = []
    for block in payload.get("data", []):
        metric   = block.get("metric")
        unit     = block.get("unit")
        interval = block.get("interval")
        for res in block.get("results", []):
            unit_code = (res.get("columns") or {}).get("unit_code")
            for ts, val in res.get("data", []):
                rows.append({"timestamp": ts, "unit_code": unit_code,
                            "metric": metric, "interval": interval, "unit": unit, "value": val})
    return rows


In [29]:
# Retrieve all facility data in batch of 5 per API call
batch_retrieval(5)

Batch 1: 1-5 already cached.
Batch 2: 6-10 already cached.
Batch 3: 11-15 already cached.
Batch 4: 16-20 already cached.
Batch 5: 21-25 already cached.
Batch 6: 26-30 already cached.
Batch 7: 31-35 already cached.
Batch 8: 36-40 already cached.
Batch 9: 41-45 already cached.
Batch 10: 46-50 already cached.
Batch 11: 51-55 already cached.
Batch 12: 56-60 already cached.
Batch 13: 61-65 already cached.
Batch 14: 66-70 already cached.
Batch 15: 71-75 already cached.
Batch 16: 76-80 already cached.
Batch 17: 81-85 already cached.
Batch 18: 86-90 already cached.
Batch 19: 91-95 already cached.
Batch 20: 96-100 already cached.
Batch 21: 101-105 already cached.
Batch 22: 106-110 already cached.
Batch 23: 111-115 already cached.
Batch 24: 116-120 already cached.
Batch 25: 121-125 already cached.
Batch 26: 126-130 already cached.
Batch 27: 131-135 already cached.
Batch 28: 136-140 already cached.
Batch 29: 141-145 already cached.
Batch 30: 146-150 already cached.
Batch 31: 151-155 already cache

## <b> <span style="color:orange">2. Data Integration and Caching</span></b>


Some facilities have more than one units, so we need to make separate tables for easier analysis.

In [30]:
# function to separate the facilities and units rows
def build_tables(facilities: list[dict]):
    facility_rows: list[dict] = []
    unit_rows: list[dict] = []

    for f in facilities:
        f_code = f.get("code")
        facility_rows.append({
            "facility_code": f_code,
            "facility_name": f.get("name"),
            "network_id": f.get("network_id"),
            "network_region": f.get("network_region"),
            "lat": (f.get("location") or {}).get("lat"),
            "lng": (f.get("location") or {}).get("lng"),
            "created_at": f.get("created_at"),
            "updated_at": f.get("updated_at"),
        })

        for u in (f.get("units") or []):
                # unify field names we care about
                unit_rows.append({
                    "unit_code": u.get("code"),
                    "facility_code": f_code,
                    "fueltech_id": u.get("fueltech_id"),
                    "status_id": u.get("status_id"),
                    "dispatch_type": u.get("dispatch_type"),
                    "capacity_registered": u.get("capacity_registered"),
                    "capacity_maximum": u.get("capacity_maximum"),
                    "capacity_storage": u.get("capacity_storage"),
                    "data_first_seen": u.get("data_first_seen"),
                    "data_last_seen": u.get("data_last_seen"),
                    "unit_created_at": u.get("created_at"),
                    "unit_updated_at": u.get("updated_at"),
                })
    
    facilities_df = pd.DataFrame(facility_rows).drop_duplicates(subset=["facility_code"]).reset_index(drop=True)
    units_lookup_df = pd.DataFrame(unit_rows).drop_duplicates(subset=["unit_code"]).reset_index(drop=True)

    return facilities_df, units_lookup_df


Save the tables into separate csv files.

In [31]:
facilities_df, units_facilities_df = build_tables(facilities['data'])

# save to csv
save_dataset(facilities_df, "DATA/EXTRACTED/electricity_facilities.csv")
save_dataset(units_facilities_df, "DATA/EXTRACTED/electricity_units_facilities.csv")

Saved: DATA\EXTRACTED\electricity_facilities.csv
Saved: DATA\EXTRACTED\electricity_units_facilities.csv


For power and emission data per facility, we need to perform some pre-processing to store them into a cached csv file. Specifically, for this process we need to:
1. Combine all cached .json data of into one dataframe.
2. Sum the facilitiy data to get total power and emissions per facility (some facilities have more than one units).
3. Append additional information to each facility (e.g. lat, lon, facility_name, etc).

In [32]:
# combine all json cache into one dataframe
records = []
for path in glob.glob("./DATA/EXTRACTED/*.json"):
    payload = json.load(open(path))
    
    records.extend(flatten(payload))

series_df = pd.DataFrame(records)

lookup = pd.read_csv("./DATA/EXTRACTED/electricity_units_facilities.csv")[["unit_code","facility_code"]]
series_df = series_df.merge(lookup, on="unit_code", how="left")
facility_df = (series_df.groupby(["timestamp","facility_code","metric"], as_index=False)["value"].sum())

facilities_df = pd.read_csv("./DATA/EXTRACTED/electricity_facilities.csv")[
    ["facility_code", "facility_name", "network_id", "network_region", "lat", "lng"]
]
facility_df = facility_df.merge(facilities_df, on="facility_code", how="left")

# reorder columns for clarity
facility_df = facility_df[
    [
        "timestamp", "facility_code", "facility_name", "network_id", "network_region",
        "lat", "lng", "metric", "value"
    ]
]


In [33]:
# save to csv
save_dataset(facility_df, "DATA/EXTRACTED/consolidated_facilities.csv")

Saved: DATA\EXTRACTED\consolidated_facilities.csv


In [34]:
df = pd.read_csv("DATA/EXTRACTED/consolidated_facilities.csv")
df

,timestamp,facility_code,facility_name,network_id,network_region,lat,lng,metric,value
0,2025-10-01T00:00:00+10:00,0MREH,Melbourne A1,NEM,VIC1,-37.661274,144.726302,emissions,0.00
1,2025-10-01T00:00:00+10:00,0MREH,Melbourne A1,NEM,VIC1,-37.661274,144.726302,power,0.00
2,2025-10-01T00:00:00+10:00,0MREHA2,Melbourne A2,NEM,VIC1,-37.663934,144.726927,emissions,0.00
3,2025-10-01T00:00:00+10:00,0MREHA2,Melbourne A2,NEM,VIC1,-37.663934,144.726927,power,0.00
4,2025-10-01T00:00:00+10:00,0TARONGBESS,Tarong,NEM,QLD1,-26.780051,151.912068,emissions,0.00
...,...,...,...,...,...,...,...,...,...
1336701,2025-10-07T23:55:00+10:00,YATSF1,Yatpool,NEM,VIC1,-34.380730,142.205340,power,0.00
1336702,2025-10-07T23:55:00+10:00,YENDONWF,Yendon,NEM,VIC1,-37.630952,144.022463,emissions,0.00
1336703,2025-10-07T23:55:00+10:00,YENDONWF,Yendon,NEM,VIC1,-37.630952,144.022463,power,89.87
1336704,2025-10-07T23:55:00+10:00,YSWF,Yaloak South,NEM,VIC1,-37.716474,144.241947,emissions,0.00


In [35]:
pivot_df = (
    df
    .pivot_table(
        index=[
            "timestamp", "facility_code", "facility_name",
            "network_id", "network_region", "lat", "lng"
        ],
        columns="metric",
        values="value"
    )
    .reset_index()
)

pivot_df.columns.name = None  # remove 'metric' label
pivot_df = pivot_df.rename_axis(None, axis=1)

In [36]:
pivot_df["timestamp"] = pd.to_datetime(pivot_df["timestamp"])
pivot_df.dtypes

timestamp         datetime64[ns, UTC+10:00]
facility_code                        object
facility_name                        object
network_id                           object
network_region                       object
lat                                 float64
lng                                 float64
emissions                           float64
power                               float64
dtype: object

In [37]:
pivot_df.head()

,timestamp,facility_code,facility_name,network_id,network_region,lat,lng,emissions,power
0,2025-10-01 00:00:00+10:00,0MREH,Melbourne A1,NEM,VIC1,-37.661274,144.726302,0.0,0.00
1,2025-10-01 00:00:00+10:00,0MREHA2,Melbourne A2,NEM,VIC1,-37.663934,144.726927,0.0,0.00
2,2025-10-01 00:00:00+10:00,0TARONGBESS,Tarong,NEM,QLD1,-26.780051,151.912068,0.0,0.00
3,2025-10-01 00:00:00+10:00,0WAMBOWF,Wambo,NEM,QLD1,-26.603045,151.246876,0.0,65.23
4,2025-10-01 00:00:00+10:00,ADP,Adelaide Desalination,NEM,SA1,-35.096948,138.484061,0.0,0.00


In [38]:
# save to csv
save_dataset(pivot_df, "DATA/EXTRACTED/consolidated_facilities_cleaned.csv")

Saved: DATA\EXTRACTED\consolidated_facilities_cleaned.csv


In [39]:
pivot_df.columns

Index(['timestamp', 'facility_code', 'facility_name', 'network_id',
       'network_region', 'lat', 'lng', 'emissions', 'power'],
      dtype='object')

## <b> <span style="color:orange">3. Data Publishing via MTQQ</span></b>


In [40]:
import mtqq_publisher as publisher

In [41]:
df = pd.read_csv("DATA/EXTRACTED/consolidated_facilities_cleaned.csv")

publisher.initialise_mqtt_client()
publisher.publish_data_stream(df)
publisher.stop_mtqq_client()

Starting to publish 668353 records...
[PUBLISHED] 0MREH @ 2025-10-01 00:00:00+10:00
[PUBLISHED] PORTLCN @ 2025-10-01 00:00:00+10:00
[PUBLISHED] PIONEER @ 2025-10-01 00:00:00+10:00
[PUBLISHED] PIBESS @ 2025-10-01 00:00:00+10:00
[PUBLISHED] PAREPW @ 2025-10-01 00:00:00+10:00
[PUBLISHED] PALOONA @ 2025-10-01 00:00:00+10:00
[PUBLISHED] OSBORNE @ 2025-10-01 00:00:00+10:00
[PUBLISHED] OAKLAND @ 2025-10-01 00:00:00+10:00
[PUBLISHED] PORTWF @ 2025-10-01 00:00:00+10:00
[PUBLISHED] OAKEY2SF @ 2025-10-01 00:00:00+10:00
[PUBLISHED] OAKEY @ 2025-10-01 00:00:00+10:00
[PUBLISHED] NYNGAN @ 2025-10-01 00:00:00+10:00
[PUBLISHED] NUMURKSF @ 2025-10-01 00:00:00+10:00
[PUBLISHED] NPPPPS @ 2025-10-01 00:00:00+10:00
[PUBLISHED] NEWPORT @ 2025-10-01 00:00:00+10:00
[PUBLISHED] NEWENSF @ 2025-10-01 00:00:00+10:00
[PUBLISHED] NEVERSF @ 2025-10-01 00:00:00+10:00
[PUBLISHED] OAKEY1SF @ 2025-10-01 00:00:00+10:00
[PUBLISHED] PSF @ 2025-10-01 00:00:00+10:00
[PUBLISHED] PTINA @ 2025-10-01 00:00:00+10:00
[PUBLISHED] QB

KeyboardInterrupt: 

## <b> <span style="color:orange">4. Dashboard and Visualisation</span></b>


In [ ]:
import mtqq_publisher as publisher

Connected with result code Success
Ready to publish to topic: COMP5339/T07G04/facilities
